In [19]:
%reset -f
import ROOT
import numpy as np
import traceback


class color:
    PURPLE    = '\033[95m'
    CYAN      = '\033[96m'
    DARKCYAN  = '\033[36m'
    BLUE      = '\033[94m'
    GREEN     = '\033[92m'
    YELLOW    = '\033[93m'
    RED       = '\033[91m'
    BOLD      = '\033[1m'
    LIGHT     = '\033[2m'
    ITALIC    = '\033[3m'
    UNDERLINE = '\033[4m'
    BLINK     = '\033[5m'
    DELTA     = '\u0394' # symbol
    END       = '\033[0m'
    
    
class color_bg:
    BLACK   = '\033[40m'
    RED     = '\033[41m'
    GREEN   = '\033[42m'
    YELLOW  = '\033[43m'
    BLUE    = '\033[44m'
    MAGENTA = '\033[45m'
    CYAN    = '\033[46m'
    WHITE   = '\033[47m'
    RESET   = '\033[49m'
    END     = '\033[0m'
    
    
class root_color:
    # Colors
    White  = 0
    Black  = 1
    Red    = 2
    Green  = 3
    Blue   = 4
    Yellow = 5
    Pink   = 6
    Cyan   = 7
    DGreen = 8 # Dark Green
    Purple = 9
    Grey   = 13
    Brown  = 28
    Gold   = 41
    Rust   = 46
    
    # Fonts
    Bold   = '#font[22]'
    Italic = '#font[12]'
    
    # Symbols
    Delta   = '#Delta'
    Phi     = '#phi'
    π       = '#pi'
    Degrees = '#circ'
    
    Line = '#splitline'


print("""
Code has started running...

Creating helpful functions...
""")


def Canvas_Create(Name="test", Num_Columns=3, Num_Rows=2, Size_X=600, Size_Y=800, cd_Space=0):
    canvas_test = ROOT.TCanvas(str(Name), str(Name), Size_X, Size_Y)
    canvas_test.Divide(Num_Columns, Num_Rows, cd_Space, cd_Space)
    canvas_test.SetGrid()
    ROOT.gStyle.SetAxisColor(16, 'xy')
    ROOT.gStyle.SetOptStat(0)
    ROOT.gStyle.SetOptFit(1)

    return canvas_test


def plot_colors(Region, Correction="Corrected", OutPut_Q="All"):
    reg_color    = root_color.Black if(Region in ["regall", "reg1", "Center"]) else root_color.Red if(Region in ["reg2", "Negative"]) else root_color.Green if(Region in ["reg3", "Positive"]) else "error"
    line_style   = 1
    marker_style = 8
    if(Correction in ["mm0", "No Correction", "Uncorrected"]):
        reg_color    = root_color.Grey if(Region in ["regall", "reg1", "Center"]) else root_color.Rust if(Region in ["reg2", "Negative"]) else root_color.DGreen if(Region in ["reg3", "Positive"]) else "error"
        line_style   = 9
        marker_style = 1
    if(reg_color == "error"):
        print(color.RED, "\nColor Error\n", color.END)
        reg_color = root_color.Black
    if(OutPut_Q == "All"):
        return [reg_color, line_style, marker_style]
    elif("color"  in OutPut_Q or "Color"  in OutPut_Q):
        return reg_color
    elif("line"   in OutPut_Q or "Line"   in OutPut_Q):
        return line_style
    elif("marker" in OutPut_Q or "Marker" in OutPut_Q):
        return marker_style
    else:
        print(color.RED, "\nOutput Error?\n", str(OutPut_Q), "\n", color.END)
        return [reg_color, line_style, marker_style]
        

def palette_move(canvas, histo, x_left, x_right, y_up, y_down):
        palette_test = 0
        canvas.Modified()
        canvas.Update()

        while(palette_test < 4 and palette_test != -1):
            try:
                palette_histo = histo.GetListOfFunctions().FindObject("palette")
                palette_histo.SetX1NDC(0.905 + x_left)
                palette_histo.SetX2NDC(0.925 + x_right)
                palette_histo.SetY1NDC(0.100 + y_down)
                palette_histo.SetY2NDC(0.900 + y_up)
                canvas.Modified()
                canvas.Update()
                palette_test = -1
            except:
                palette_test += 1

        if(palette_test > 0):
            print("\nFailed to move palette...")
              
    
def Draw_Canvas(canvas, cd_num=1, left_add=0, right_add=0, up_add=0, down_add=0):
    canvas.cd(cd_num)
    canvas.cd(cd_num).SetLeftMargin(  0.05 + left_add)
    canvas.cd(cd_num).SetRightMargin( 0.05 + right_add)
    canvas.cd(cd_num).SetTopMargin(   0.10 + up_add)
    canvas.cd(cd_num).SetBottomMargin(0.10 + down_add)
    
    
def statbox_move(Histogram, Canvas, Default_Stat_Obj, Sector=1, Print_Method="norm", Y1_add=0, Y2_add=0, X1_add=0, X2_add=0):
    finding, search = 0, 0
    Canvas.Modified()
    Canvas.Update()

    while(finding == 0 and search < 5):
        if(Default_Stat_Obj == ""):
            Default_Stat_Obj = Histogram.GetListOfFunctions().FindObject("stats")
        if("TPaveStats" not in str(type(Default_Stat_Obj))):
            try:
                Default_Stat_Obj = Histogram.GetListOfFunctions().FindObject("stats")
            except Exception as e:
                print(color.RED, str(e), color.END)
        try:
            if(Print_Method == "MM_1D"):
                Default_Stat_Obj.SetY1NDC(0.125)
                # Default_Stat_Obj.SetY2NDC(0.625)
                Default_Stat_Obj.SetY2NDC(0.605)
                Default_Stat_Obj.SetX1NDC(0.125)
                Default_Stat_Obj.SetX2NDC(0.425)
            
            if(Print_Method == "off"):
                Default_Stat_Obj.SetY1NDC(0)
                Default_Stat_Obj.SetY2NDC(0)
                Default_Stat_Obj.SetX1NDC(0)
                Default_Stat_Obj.SetX2NDC(0)
            
            if(Print_Method == "norm"):
                Default_Stat_Obj.SetY1NDC(0.05 + Y1_add)
                Default_Stat_Obj.SetY2NDC(0.25 + Y2_add)
                Default_Stat_Obj.SetX1NDC(0.15 + X1_add)
                Default_Stat_Obj.SetX2NDC(0.45 + X2_add)
            
            if(Print_Method == "ver"):
                Default_Stat_Obj.SetY1NDC(0.05 + Y1_add)
                Default_Stat_Obj.SetY2NDC(0.25 + Y2_add)
                
                if(Sector != -1):
                    if(Sector > 4):
                        Default_Stat_Obj.SetY1NDC(0.15 + Y1_add)
                        Default_Stat_Obj.SetY2NDC(0.25 + Y2_add)

                    if(Sector%2 == 0):
                        Default_Stat_Obj.SetX1NDC(0.05 + X1_add)
                        Default_Stat_Obj.SetX2NDC(0.35 + X2_add)
                    else:
                        Default_Stat_Obj.SetX1NDC(0.15 + X1_add)
                        Default_Stat_Obj.SetX2NDC(0.45 + X2_add)

            if(Print_Method == "hor"):
                Default_Stat_Obj.SetY1NDC(0.05 + Y1_add)
                Default_Stat_Obj.SetY2NDC(0.25 + Y2_add)
                
                if(Sector != -1):
                    if(Sector > 3):
                        Default_Stat_Obj.SetY1NDC(0.15 + Y1_add)
                        Default_Stat_Obj.SetY2NDC(0.35 + Y2_add)

                    if(Sector != 1 and Sector != 4):
                        Default_Stat_Obj.SetX1NDC(0.05 + X1_add)
                        Default_Stat_Obj.SetX2NDC(0.35 + X2_add)
                    else:
                        Default_Stat_Obj.SetX1NDC(0.15 + X1_add)
                        Default_Stat_Obj.SetX2NDC(0.45 + X2_add)

            Default_Stat_Obj.Draw("same")
            
            Canvas.Modified()
            Canvas.Update()
            
            finding += 1
            
        except Exception as e:
            Canvas.Modified()
            Canvas.Update()
            
            if(search > 2):
                print("".join([color.RED, "Search is Failing... (", str(search),")", color.END]))
                print("Error: ", str(e))
                fail
            finding = 0
            search += 1

    if(search > 4):
        print("Failed search")

def print_rounded_str(number, rounding=0):
    try:
        #if(rounding != 0 and type(number) is not str and (abs(number) > 10**(-abs(rounding-round_science)) or abs(number) >= 0.001)):
        if(rounding != 0 and abs(number) >= 0.001):
            output = round(number, rounding)
            output = "".join(["{:.", str(rounding),    "}"]).format(number)
            # print("round")
        elif(rounding != 0):
            output = "".join(["{:.", str(rounding-1), "e}"]).format(number)
            # print("science")
        else:
            # print("other")
            output = number
            
        return output
    
    except Exception as e:
        print("".join([color.BOLD, color.RED, "Error: number = ", str(output), " is not accepted", " --> failed to round input..." if(rounding != 0) else "", "\nERROR Output Is: \n", str(e), color.END]))
        return number
    
    

    
print("DONE")

Welcome to JupyROOT 6.22/06

Code has started running...

Creating helpful functions...

DONE


In [ ]:
##### print(color.BOLD + "Sliced New corrections..." + color.END)
correction_Pars, pp_List = [], []

def print_con(sector=1):
#     return False
    return True
    if(sector in [4]):
        return True
    else:
        return False
    
    return False
        
    

print_as_python = True
print_as_python = False
    
inc_p = 0.15
# inc_p = 0.075
pp_min = 0.45
pp_max = 3.0 + inc_p
while((pp_min+inc_p) < pp_max):
    pp_List.append(round(pp_min, 4))
    # print(pp_List)
    pp_min += inc_p

for sec in range(1, 6+1, 1):
    if(print_con(sec)):
        print(str("".join([color.BOLD, "if(sec == ", str(sec), "):", color.END])) if(print_as_python) else str("".join([color.BOLD, "if(sec == ", str(sec), "){", color.END])))
    ii = 0
    for pp in pp_List:
        ii += 1
        if(ii == len(pp_List)):
            continue
        pp += 0.5*inc_p
        dp = 0
        if(print_con(sec)):
            print(str("".join(["""    # Peak #""", str(ii), """:
    if(""", str(pp_List[ii - 1]) ," < pp and pp < ", str(pp_List[ii]) , "):"])) if(print_as_python) else str("".join(["""    // Peak #""", str(ii), """:
    if(""", str(pp_List[ii - 1]) ," < pp && pp < ", str(pp_List[ii]) , "){"])))




        mom_X = (pp_List[ii] + pp_List[ii - 1])/2
        cor_Y = round(dp, 5)
        
        correction_Pars.append([mom_X, cor_Y, str(sec)])

        if(print_con(sec)):        
            print(str("".join(["        dp = (", str(print_rounded_str(dp, rounding=5)), ")"])) if(print_as_python) else str("".join(["        dp = (", str(print_rounded_str(dp, rounding=5)), ");\n    }"])))
    if(print_con(sec)):
        print("\n" if(print_as_python) else "}\n")
    
    
#         print("".join(["For pp = ", str(print_rounded_str(pp, rounding=5)), ", the correction factor is ", color.BOLD, "dp = ", str(print_rounded_str(dp, rounding=5)), color.END]))


print("\n\n")
Dp_Mom, Canvas_Dp = {}, {}

Original_Cor_Lines, New_Cor_Lines = {}, {}

min_Fit = 2.25
max_Fit = 9.5




for sec in range(1, 7, 1):
    Dp_Mom[str(sec)] = ROOT.TGraphErrors("".join(["Correction Parameters for Sector ", str(sec), ";pp;dp"]), "".join(["Correction Parameters for Sector ", str(sec), ";pp;dp"]))
    Dp_Mom[str(sec)].SetTitle("".join(["Correction Parameters for Sector ", str(sec), ";pp;dp"]))
    Dp_Mom[str(sec)].SetMarkerStyle(20)
    Dp_Mom[str(sec)].SetMarkerColor(4)
    Dp_Mom[str(sec)].SetLineColor(4)


for points in correction_Pars:
    mom_X, cor_Y, sec = points
    Dp_Mom[str(sec)].SetPoint(Dp_Mom[str(sec)].GetN(), mom_X, cor_Y)
    
    
sec = "all"
Canvas_Dp[str(sec)] = Canvas_Create(Name="".join(["Correction Parameters for Sector ", str(sec)]), Num_Columns=3, Num_Rows=2, Size_X=1200, Size_Y=800, cd_Space=0)
Canvas_Dp[str(sec)].Draw()
for sec in range(1, 6+1, 1):
    Draw_Canvas(canvas=Canvas_Dp[str("all")], cd_num=sec, left_add=0.1, right_add=0, up_add=0, down_add=0)
    Dp_Mom[str(sec)].Draw("APL same")
    
    fitting_line = ROOT.TF1("pol2", "".join(["((1 + TMath::Sign(1, -(x - ", str(switch_point), ")))/2)*([pf2]*x*x + [pf1]*x + [pf0]) + ((1 + TMath::Sign(1, (x - ", str(switch_point), ")))/2)*([ps2]*(x - ", str(switch_point), ")*(x - ", str(switch_point), ") + [ps1]*(x - ", str(switch_point), ") + ([pf2]*", str(switch_point), "*", str(switch_point), " + [pf1]*", str(switch_point), " + [pf0]))"]), min_Fit, max_Fit)
    
    
#     if(sec == 5):
#         fitting_line = ROOT.TF1("pol2_then_pol2", "".join(["[pf2]*x*x + [pf1]*x + [pf0]"]), min_Fit, 1.125)
# #         fitting_line = ROOT.TF1("pol2_then_pol2", "".join(["[pf2]*x*x + [pf1]*x + [pf0]"]), 1.425, max_Fit)
# #         fitting_line = ROOT.TF1("pol2_then_pol2", "".join(["((1 + TMath::Sign(1, -(x - ", str(switch_point), ")))/2)*((0.02597)*x*x + (-0.04597)*x + (0.03339)) + ((1 + TMath::Sign(1, (x - ", str(switch_point), ")))/2)*([ps2]*(x - ", str(switch_point), ")*(x - ", str(switch_point), ") + [ps1]*(x - ", str(switch_point), ") + ((0.02597)*", str(switch_point), "*", str(switch_point), " + (-0.04597)*", str(switch_point), " + (0.03339)))"]), min_Fit, max_Fit)
    
    fitting_line.SetLineColor(3)

    
    switch_point_num = 1.0 if(sec in [3]) else 1.1 if(sec in [5, 6]) else 1.38 if(sec in [1]) else 0.95
    switch_point_range = 1.2 if(sec in [5]) else 0.05 if(sec in [2]) else 0.25
    
#                                                                                                                                                1.075-0.075
    switch_point_num    = 1.5  if(sec == 1) else 0.825 if(sec == 2) else 0.975 if(sec == 3) else 1.125 if(sec == 4) else 1.125 if(sec == 5) else 0.975 if(sec == 6) else 1.5
    switch_point_range  = 0.15 if(sec == 1) else 0.25  if(sec == 2) else 0.05  if(sec == 3) else 0.025 if(sec == 4) else 0.25  if(sec == 5) else 0.125 if(sec == 6) else 1.5
    
    if(sec == 5):
        switch_point_num   = 1.125 # 0.975# 1.125 + 2*0.15
        switch_point_range = 0.5
    
#     if(sec != 5):
    fitting_line.SetParName(5, "Switch Point")
    fitting_line.SetParameter(5, switch_point_num)
    fitting_line.SetParLimits(5, switch_point_num - switch_point_range, switch_point_num + switch_point_range)
#     else:
#         fitting_line.SetParName(2, "Switch Point")
#         fitting_line.SetParameter(2, switch_point_num)
#         fitting_line.SetParLimits(2, switch_point_num - switch_point_range, switch_point_num + switch_point_range)
        
    
# #     fit_exp_line = ROOT.TF1("ELoss_Mom", "exp([ep0] + [ep1]*x) + [p0]", min_Fit, max_Fit)
#     fit_exp_line = ROOT.TF1("ELoss_Mom", "exp([ep0] + [ep1]*x) + [p0]", switch_point_num, max_Fit)
#     fit_exp_line.SetLineColor(2)
    
#     Dp_Mom[str(sec)].Fit(fit_exp_line, "RQ")
    Dp_Mom[str(sec)].Fit(fitting_line, "RQ+")
    
    
    Original_Cor_Lines["".join(["(CorPro==15) Sec_", str(sec)])].SetLineColor(5)
    Original_Cor_Lines["".join(["(CorPro==15) Sec_", str(sec)])].Draw("same")
    
# #     Original_Cor_Lines[str(sec)].SetLineColor(3)
# #     Original_Cor_Lines[str(sec)].Draw("same")
#     Original_Cor_Lines["".join(["New_Failed_", str(sec)])].SetLineColor(2)
#     Original_Cor_Lines["".join(["New_Failed_", str(sec)])].Draw("same")
    

    
    try:
        par_pf2 = round(Dp_Mom[str(sec)].GetFunction("pol2_then_pol2").GetParameter("pf2"), 8)
        if(abs(par_pf2) < 0.01):
            par_pf2 = "{:.4e}".format(par_pf2)
        else:
            par_pf2 = round(par_pf2, 5)

        par_pf1 = round(Dp_Mom[str(sec)].GetFunction("pol2_then_pol2").GetParameter("pf1"), 8)
        if(abs(par_pf1) < 0.01):
            par_pf1 = "{:.4e}".format(par_pf1)
        else:
            par_pf1 = round(par_pf1, 5)

        par_pf0 = round(Dp_Mom[str(sec)].GetFunction("pol2_then_pol2").GetParameter("pf0"), 8)
        if(abs(par_pf0) < 0.01):
            par_pf0 = "{:.4e}".format(par_pf0)
        else:
            par_pf0 = round(par_pf0, 5)

        par_ps2 = round(Dp_Mom[str(sec)].GetFunction("pol2_then_pol2").GetParameter("ps2"), 8)
        if(abs(par_ps2) < 0.01):
            par_ps2 = "{:.4e}".format(par_ps2)
        else:
            par_ps2 = round(par_ps2, 5)

        par_ps1 = round(Dp_Mom[str(sec)].GetFunction("pol2_then_pol2").GetParameter("ps1"), 8)
        if(abs(par_ps1) < 0.01):
            par_ps1 = "{:.4e}".format(par_ps1)
        else:
            par_ps1 = round(par_ps1, 5)

        switch_point = round(Dp_Mom[str(sec)].GetFunction("pol2_then_pol2").GetParameter("Switch Point"), 8)
        if(abs(switch_point) < 0.01):
            switch_point = "{:.4e}".format(switch_point)
        else:
            switch_point = round(switch_point, 5)

        print("".join(["""if(sec == """, str(sec) , """){
        dp = ((1 + TMath::Sign(1, -(pp - (""", str(switch_point), "))))/2)*((", str(par_pf2), ")*pp*pp + (", str(par_pf1), ")*pp + (", str(par_pf0), ")) + ((1 + TMath::Sign(1, (pp - (", str(switch_point), "))))/2)*((", str(par_ps2), ")*(pp - (", str(switch_point), "))*(pp - (", str(switch_point), ")) + (", str(par_ps1), ")*(pp - (", str(switch_point), ")) + ((", str(par_pf2), ")*(", str(switch_point), ")*(", str(switch_point), ") + (", str(par_pf1), ")*(", str(switch_point), ") + (", str(par_pf0), """)));
    }"""]))
    #     print("".join(["""if(sec == """, str(sec) , """){
    #     dp = ((1 + TMath::Sign(1, -(pp - (""", str(switch_point), "))))/2)*((", str(par_pf2), ")*pp*pp + (", str(par_pf1), ")*pp + (", str(par_pf0), ")) + ((1 + TMath::Sign(1, (pp - (", str(switch_point), "))))/2)*((", str(par_ps2), ")*(pp - (", str(switch_point), "))*(pp - (", str(switch_point), ")) + (", str(par_ps1), ")*(pp - (", str(switch_point), ")) + (", str(round(par_pf2*switch_point*switch_point + par_pf1*switch_point + par_pf0, 6)), """));
    # }"""]))
    
    except Exception as e:
        print("".join([color.BOLD, color.RED, "ERROR: ", str(e), color.END]))

    #     print("".join(["""
    # ((1 + TMath::Sign(1, -(x - (""", str(switch_point), "))))/2)*((", str(par_pf2), ")*x*x + (", str(par_pf1), ")*x + (", str(par_pf0), ")) + ((1 + TMath::Sign(1, (x - (", str(switch_point), "))))/2)*((", str(par_ps2), ")*(x - (", str(switch_point), "))*(x - (", str(switch_point), ")) + (", str(par_ps1), ")*(x - (", str(switch_point), ")) + ((", str(par_pf2), ")*(", str(switch_point), ")*(", str(switch_point), ") + (", str(par_pf1), ")*(", str(switch_point), ") + (", str(par_pf0), """)))"""]))

print("\n\n")
# print(""""((1 + TMath::Sign(1, -(x - ", [switch_point], ")))/2)*([pf2]*x*x + [pf1]*x + [pf0]) + ((1 + TMath::Sign(1, (x - ", [switch_point], ")))/2)*([ps2]*(x - ", [switch_point], ")*(x - ", [switch_point], ") + [ps1]*(x - ", [switch_point], ") + ([pf2]*", [switch_point], "*", [switch_point], " + [pf1]*", [switch_point], " + [pf0]))""")


In [ ]:
"""

# The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = No Correction][Sector 1] is:
dp =       ((5.0405e-06)*phi*phi +  (3.8245e-05)*phi +  (-0.0013668))*pp*pp + ((-6.3680e-05)*phi*phi + (-3.5346e-05)*phi +    (0.017592))*pp +  ((1.7893e-04)*phi*phi +  (4.9969e-04)*phi + (-0.034336))
                                                                                                                                                                                                                                
# The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Without Pion)][Sector 1] is:
dp = dp + ((-9.1565e-06)*phi*phi + (-3.0121e-05)*phi +  (1.0958e-04))*pp*pp +  ((1.2296e-04)*phi*phi +  (4.7107e-04)*phi +  (-0.0013516))*pp + ((-3.6182e-04)*phi*phi +  (-0.0014695)*phi + (0.0054485))
                                                                                                                                                                                                                                
# The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Without Pion)][Sector 1] is:
dp = dp +  ((1.2011e-05)*phi*phi + (-1.0377e-06)*phi + (-1.0079e-04))*pp*pp + ((-1.6154e-04)*phi*phi +  (3.4794e-05)*phi +  (8.4711e-04))*pp +  ((4.8920e-04)*phi*phi + (-1.8913e-04)*phi + (-8.0855e-04))
                                                                                                                                                                                                                                
# The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Without Pion)][Sector 1] is:
dp = dp + ((-1.0435e-05)*phi*phi +  (1.4137e-05)*phi +  (3.9998e-05))*pp*pp +  ((1.4518e-04)*phi*phi + (-1.5993e-04)*phi + (-2.3791e-04))*pp + ((-4.5680e-04)*phi*phi +  (2.6327e-04)*phi + (6.0975e-05))
                                                                                                                                                                                                                                
# The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Without Pion)][Sector 1] is:
dp = dp +  ((9.6177e-06)*phi*phi + (-1.6643e-06)*phi + (-7.5806e-05))*pp*pp + ((-1.3124e-04)*phi*phi + (-1.1399e-05)*phi +  (5.9901e-04))*pp +  ((4.1151e-04)*phi*phi +  (2.2676e-04)*phi + (-0.0012323))
                                                                                                                                                                                                                                
# The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Without Pion)][Sector 1] is:
dp = dp + ((-1.5949e-05)*phi*phi +  (4.1737e-06)*phi +  (9.3051e-04))*pp*pp +  ((2.2274e-04)*phi*phi + (-5.5908e-05)*phi +   (-0.012478))*pp + ((-7.2884e-04)*phi*phi +  (2.3293e-04)*phi + (0.039049))
                                                                                                                                                                                                                                
# The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Without Pion)][Sector 1] is:
dp = dp +  ((1.2672e-05)*phi*phi + (-7.2349e-06)*phi + (-3.3866e-04))*pp*pp + ((-1.9480e-04)*phi*phi +  (1.1734e-04)*phi +   (0.0048881))*pp +  ((7.2094e-04)*phi*phi + (-4.9098e-04)*phi + (-0.017591))
                                                                                                                                                                                                                                
# The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Without Pion)][Sector 1] is:
dp = dp + ((-1.6711e-05)*phi*phi + (-5.0132e-06)*phi +  (4.3982e-04))*pp*pp +  ((2.6100e-04)*phi*phi +  (5.5687e-05)*phi +  (-0.0069082))*pp + ((-9.7378e-04)*phi*phi + (-8.6253e-05)*phi + (0.025668))
                                                                                                                                                                                                                                
# The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Without Pion)][Sector 1] is:
dp = dp +  ((1.5606e-05)*phi*phi +  (2.6339e-06)*phi + (-4.9270e-04))*pp*pp + ((-2.4409e-04)*phi*phi + (-5.0027e-05)*phi +   (0.0071274))*pp +  ((9.1998e-04)*phi*phi +  (2.0150e-04)*phi + (-0.024643))
                                                                                                                                                                                                                                
    
"""

Aa = (5.0405e-06) + (-9.1565e-06) + (1.2011e-05) + (-1.0435e-05) + (9.6177e-06) + (-1.5949e-05) + (1.2672e-05) + (-1.6711e-05) + (1.5606e-05)

In [8]:
# import re

# def extract_parameters(eq):
#     # Define a regular expression pattern to match the parameter values
#     pattern = r"[-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?"
#     # Use the pattern to find all the parameter values in the equation
#     params = re.findall(pattern, eq)
#     # Convert the parameter values to floats and return them
#     return [float(p) for p in params]


# equation = "dp = dp + ((-9.1565e-06)*phi*phi + (-3.0121e-05)*phi +  (1.0958e-04))*pp*pp +  ((1.2296e-04)*phi*phi +  (4.7107e-04)*phi +  (-0.0013516))*pp + ((-3.6182e-04)*phi*phi +  (-0.0014695)*phi + (0.0054485))"
# #          "dp = dp + ((-9.1565e-06)*phi*phi + (-3.0121e-05)*phi +  (0.00010958)*pp*pp + ((0.00012296)*phi*phi + (0.00047107)*phi + (-0.0013516))*pp + ((-0.00036182)*phi*phi + (-0.0014695)*phi + (0.0054485)); "
# #          "dp = dp + ((-9.1565e-06)*phi*phi + (-3.0121e-05)*phi +  (1.0958e-04))*pp*pp +  ((1.2296e-04)*phi*phi +  (4.7107e-04)*phi +  (-0.0013516))*pp + ((-3.6182e-04)*phi*phi +  (-0.0014695)*phi + (0.0054485));" 
# equation = "dp = dp + ((0)*phi*phi + (0)*phi + (0))*pp*pp + ((0)*phi*phi + (0)*phi + (0))*pp + ((0)*phi*phi + (0)*phi + (0));"
# # coefficients = []

# # # Use regular expressions to find all the coefficients in the equation
# # pattern = r'[-+]?\d*\.\d+'
# # matches = re.findall(pattern, equation)

# # # Convert the strings to floats and store them in the coefficients list
# # for match in matches:
# #     coefficients.append(float(match))

# # print(coefficients)

# para_terms = extract_parameters(equation)
# print(extract_parameters(equation))

# # Sector 1
# eqs_list = ["dp =       ((5.0405e-06)*phi*phi +  (3.8245e-05)*phi +  (-0.0013668))*pp*pp + ((-6.3680e-05)*phi*phi + (-3.5346e-05)*phi +    (0.017592))*pp +  ((1.7893e-04)*phi*phi +  (4.9969e-04)*phi + (-0.034336));", "dp = dp + ((-9.1565e-06)*phi*phi + (-3.0121e-05)*phi +  (1.0958e-04))*pp*pp +  ((1.2296e-04)*phi*phi +  (4.7107e-04)*phi +  (-0.0013516))*pp + ((-3.6182e-04)*phi*phi +  (-0.0014695)*phi +   (0.0054485));", "dp = dp +  ((1.2011e-05)*phi*phi + (-1.0377e-06)*phi + (-1.0079e-04))*pp*pp + ((-1.6154e-04)*phi*phi +  (3.4794e-05)*phi +  (8.4711e-04))*pp +  ((4.8920e-04)*phi*phi + (-1.8913e-04)*phi + (-8.0855e-04));", "dp = dp + ((-1.0435e-05)*phi*phi +  (1.4137e-05)*phi +  (3.9998e-05))*pp*pp +  ((1.4518e-04)*phi*phi + (-1.5993e-04)*phi + (-2.3791e-04))*pp + ((-4.5680e-04)*phi*phi +  (2.6327e-04)*phi +  (6.0975e-05));", "dp = dp +  ((9.6177e-06)*phi*phi + (-1.6643e-06)*phi + (-7.5806e-05))*pp*pp + ((-1.3124e-04)*phi*phi + (-1.1399e-05)*phi +  (5.9901e-04))*pp +  ((4.1151e-04)*phi*phi +  (2.2676e-04)*phi + (-0.0012323));", "dp = dp + ((-1.5949e-05)*phi*phi +  (4.1737e-06)*phi +  (9.3051e-04))*pp*pp +  ((2.2274e-04)*phi*phi + (-5.5908e-05)*phi +   (-0.012478))*pp + ((-7.2884e-04)*phi*phi +  (2.3293e-04)*phi + (0.039049));", "dp = dp +  ((1.2672e-05)*phi*phi + (-7.2349e-06)*phi + (-3.3866e-04))*pp*pp + ((-1.9480e-04)*phi*phi +  (1.1734e-04)*phi +   (0.0048881))*pp +  ((7.2094e-04)*phi*phi + (-4.9098e-04)*phi + (-0.017591));", "dp = dp + ((-1.6711e-05)*phi*phi + (-5.0132e-06)*phi +  (4.3982e-04))*pp*pp +  ((2.6100e-04)*phi*phi +  (5.5687e-05)*phi +  (-0.0069082))*pp + ((-9.7378e-04)*phi*phi + (-8.6253e-05)*phi + (0.025668));", "dp = dp +  ((1.5606e-05)*phi*phi +  (2.6339e-06)*phi + (-4.9270e-04))*pp*pp + ((-2.4409e-04)*phi*phi + (-5.0027e-05)*phi +   (0.0071274))*pp +  ((9.1998e-04)*phi*phi +  (2.0150e-04)*phi + (-0.024643));"]
# # eqs_list = ["dp = dp + ((0.2)*phi*phi + (0.2)*phi + (0.2))*pp*pp + ((0.2)*phi*phi + (0.2)*phi + (0.2))*pp + ((0.2)*phi*phi + (0.2)*phi + (0.2));", "dp = dp + ((0.2)*phi*phi + (0.2)*phi + (0.2))*pp*pp + ((0.2)*phi*phi + (0.2)*phi + (0.2))*pp + ((0.2)*phi*phi + (0.2)*phi + (0.2));", "dp = dp + ((-9.1565e-06)*phi*phi + (-3.0121e-05)*phi +  (1.0958e-04))*pp*pp +  ((1.2296e-04)*phi*phi +  (4.7107e-04)*phi +  (-0.0013516))*pp + ((-3.6182e-04)*phi*phi +  (-0.0014695)*phi + (0.0054485))"]
# eqs_list = ["dp =       ((1.1615e-05)*phi*phi + (-3.0824e-05)*phi + (-4.0009e-04))*pp*pp + ((-1.7698e-04)*phi*phi +  (3.2847e-04)*phi +   (0.0055364))*pp +  ((6.7270e-04)*phi*phi + (-4.9112e-04)*phi + (-0.019513))", "dp = dp + ((-3.1457e-06)*phi*phi +  (3.3069e-05)*phi +  (5.2678e-05))*pp*pp +  ((5.5524e-05)*phi*phi + (-4.1091e-04)*phi + (-9.5712e-04))*pp + ((-2.4307e-04)*phi*phi +   (0.0010894)*phi + (0.0053037))", "dp = dp +  ((1.8553e-06)*phi*phi +  (7.9292e-09)*phi + (-1.8541e-04))*pp*pp + ((-2.4481e-05)*phi*phi + (-8.3918e-08)*phi +    (0.002448))*pp +  ((7.5016e-05)*phi*phi +  (3.8513e-07)*phi + (-0.0075036))", "dp = dp +  ((5.0720e-10)*phi*phi + (-1.5999e-09)*phi +  (2.5782e-08))*pp*pp + ((-1.5152e-08)*phi*phi +  (7.2717e-08)*phi + (-9.6885e-07))*pp +  ((3.9400e-08)*phi*phi + (-2.8714e-07)*phi + (3.6082e-06))"]

# # Sector 2
# eqs_list = ["dp =       ((1.0601e-06)*phi*phi + (-2.0599e-05)*phi + (-9.3989e-04))*pp*pp + ((-2.9184e-05)*phi*phi +  (6.4713e-04)*phi +    (0.013571))*pp +  ((1.2782e-04)*phi*phi +  (-0.0015982)*phi + (-0.035799));", "dp = dp +  ((1.8037e-06)*phi*phi +  (5.8295e-05)*phi + (-6.4276e-04))*pp*pp + ((-2.2497e-05)*phi*phi + (-7.2470e-04)*phi +     (0.00799))*pp +  ((6.4252e-05)*phi*phi +   (0.0020054)*phi + (-0.019967));", "dp = dp + ((-5.3639e-06)*phi*phi + (-3.9588e-05)*phi +  (2.5821e-04))*pp*pp +  ((7.1482e-05)*phi*phi +  (5.6509e-04)*phi +  (-0.0029168))*pp + ((-1.9994e-04)*phi*phi +  (-0.0017939)*phi + (0.0049078));", "dp = dp +  ((7.3134e-06)*phi*phi +  (2.1578e-07)*phi +  (8.8102e-05))*pp*pp + ((-1.0626e-04)*phi*phi + (-1.6914e-05)*phi + (-7.7181e-04))*pp +  ((3.3884e-04)*phi*phi +  (1.5723e-04)*phi + (0.0021254));", "dp = dp + ((-6.5403e-06)*phi*phi +  (1.8051e-05)*phi + (-1.5816e-04))*pp*pp +  ((9.7305e-05)*phi*phi + (-2.3155e-04)*phi +   (0.0016089))*pp + ((-3.0901e-04)*phi*phi +  (6.0455e-04)*phi + (-0.003992));", "dp = dp + ((-4.0123e-06)*phi*phi + (-2.8548e-05)*phi +   (0.0010347))*pp*pp +  ((5.2701e-05)*phi*phi +  (4.0875e-04)*phi +   (-0.013862))*pp + ((-1.7368e-04)*phi*phi +  (-0.0013278)*phi + (0.042685));"]
# eqs_list = ["dp =      ((-5.6520e-06)*phi*phi +  (2.3545e-05)*phi + (-3.1333e-05))*pp*pp +  ((8.2658e-05)*phi*phi + (-3.3190e-04)*phi +   (0.0011355))*pp + ((-2.7514e-04)*phi*phi +   (0.0010378)*phi + (-0.008525))", "dp = dp +  ((2.1621e-06)*phi*phi + (-1.6806e-05)*phi + (-3.5729e-05))*pp*pp + ((-2.3385e-05)*phi*phi +  (2.3059e-04)*phi + (-1.7393e-04))*pp +  ((3.4600e-05)*phi*phi + (-6.8877e-04)*phi + (0.0061243))", "dp = dp +  ((4.2842e-09)*phi*phi + (-2.3696e-09)*phi + (-1.9989e-07))*pp*pp + ((-4.2604e-08)*phi*phi +  (2.9996e-08)*phi +  (2.4800e-06))*pp +  ((9.3148e-08)*phi*phi + (-6.5076e-09)*phi + (-7.4998e-06))"]

# # Sector 3
# eqs_list = ["dp =       ((4.0477e-06)*phi*phi + (-1.1090e-05)*phi + (-8.8179e-04))*pp*pp + ((-7.5277e-05)*phi*phi +  (1.1253e-05)*phi + (0.015032))*pp + ((3.2857e-04)*phi*phi + (-1.4871e-04)*phi + (-0.03637));", "dp = dp + ((-1.2961e-05)*phi*phi + (-5.0593e-06)*phi + (-2.0861e-05))*pp*pp +  ((2.0698e-04)*phi*phi + (-1.1223e-05)*phi + (-0.0015348))*pp + ((-7.8680e-04)*phi*phi + (4.1276e-04)*phi + (0.016077));", "dp = dp +  ((1.1994e-05)*phi*phi +  (4.8958e-06)*phi + (-2.9240e-04))*pp*pp + ((-1.9392e-04)*phi*phi + (-3.9522e-05)*phi + (0.0049301))*pp + ((7.6922e-04)*phi*phi + (1.2448e-05)*phi + (-0.021559));", "dp = dp + ((-7.9820e-06)*phi*phi +  (7.7508e-07)*phi + (-4.4541e-05))*pp*pp +  ((1.2196e-04)*phi*phi +  (3.6743e-06)*phi + (0.001536))*pp + ((-4.6393e-04)*phi*phi + (-8.7269e-05)*phi + (-0.0082336));", "dp = dp +  ((3.0603e-07)*phi*phi + (-1.5040e-05)*phi +  (6.0727e-04))*pp*pp + ((-1.0623e-05)*phi*phi +  (1.7486e-04)*phi +  (-0.008958))*pp +  ((8.0536e-05)*phi*phi + (-3.9857e-04)*phi + (0.030768));", "dp = dp + ((-2.4662e-07)*phi*phi + (-2.2071e-06)*phi +  (4.5111e-05))*pp*pp +  ((4.9086e-06)*phi*phi +  (3.4926e-05)*phi + (-3.7982e-04))*pp + ((-2.4268e-05)*phi*phi + (-1.5430e-04)*phi + (-5.1332e-04));"]
# eqs_list = ["dp =      ((-5.5929e-06)*phi*phi + (-5.1052e-05)*phi + (1.1119e-04))*pp*pp + ((8.0809e-05)*phi*phi +  (6.5723e-04)*phi + (-0.0011873))*pp + ((-2.2746e-04)*phi*phi + (-0.0016036)*phi + (-6.7620e-04))", "dp = dp + ((-9.6036e-07)*phi*phi +  (2.6289e-05)*phi + (1.4462e-04))*pp*pp + ((9.9967e-06)*phi*phi + (-3.3654e-04)*phi + (-0.0019827))*pp + ((-5.5851e-05)*phi*phi + (7.1180e-04)*phi +   (0.0097756))"]


# # Sector 4
# eqs_list = ["dp = ((-2.8052e-06)*phi*phi + (-2.2854e-05)*phi + (-4.1159e-04))*pp*pp + ((3.3257e-05)*phi*phi + (1.9861e-04)*phi + (0.0050248))*pp + ((-3.7414e-05)*phi*phi + (-6.1781e-04)*phi + (1.4903e-04));", "dp = dp + ((8.1342e-06)*phi*phi + (4.0946e-05)*phi + (-0.0010942))*pp*pp + ((-9.8339e-05)*phi*phi + (-5.2091e-04)*phi + (0.014025))*pp + ((2.1987e-04)*phi*phi + (0.0013168)*phi + (-0.034387));", "dp = dp + ((-1.6781e-05)*phi*phi + (-4.9127e-05)*phi + (0.0011043))*pp*pp + ((2.1672e-04)*phi*phi + (5.7714e-04)*phi + (-0.014288))*pp + ((-5.7731e-04)*phi*phi + (-0.0012425)*phi + (0.039549));", "dp = dp + ((1.6136e-05)*phi*phi + (4.5230e-06)*phi + (-2.7187e-04))*pp*pp + ((-2.1837e-04)*phi*phi + (-5.4798e-05)*phi + (0.0044954))*pp + ((6.2835e-04)*phi*phi + (4.7089e-05)*phi + (-0.016063));", "dp = dp + ((-9.8782e-06)*phi*phi +  (1.4314e-05)*phi + (-4.5577e-04))*pp*pp +  ((1.2864e-04)*phi*phi + (-1.1600e-04)*phi +   (0.0057963))*pp + ((-3.4841e-04)*phi*phi +  (2.4823e-05)*phi + (-0.0175));", "dp = dp + ((1.9571e-06)*phi*phi + (-4.0049e-05)*phi + (5.8303e-04))*pp*pp + ((-2.5271e-05)*phi*phi + (5.1827e-04)*phi + (-0.0073137))*pp + ((6.7075e-05)*phi*phi + (-0.0014843)*phi + (0.019712));", "dp = dp +  ((4.1578e-06)*phi*phi +  (2.0446e-05)*phi + (-4.4732e-04))*pp*pp + ((-4.2237e-05)*phi*phi + (-2.6439e-04)*phi +  (0.0050143))*pp + ((6.6987e-05)*phi*phi +  (6.2765e-04)*phi +  (-0.01015));"]
# eqs_list = ["dp =      ((-7.4614e-06)*phi*phi +  (3.0297e-05)*phi +  (9.1086e-04))*pp*pp +  ((1.0988e-04)*phi*phi + (-4.7922e-04)*phi + (-0.012592))*pp + ((-3.5261e-04)*phi*phi +  (0.0016901)*phi +  (0.037815))", "dp = dp +  ((4.5434e-06)*phi*phi + (-1.7239e-05)*phi + (-8.0322e-04))*pp*pp + ((-6.5042e-05)*phi*phi +  (2.9928e-04)*phi +  (0.011107))*pp +  ((2.0436e-04)*phi*phi + (-0.0011498)*phi + (-0.033574))"]


# # Sector 5
# eqs_list = ["dp = ((-2.1116e-06)*phi*phi + (1.6136e-05)*phi + (-0.00135))*pp*pp + ((7.6007e-06)*phi*phi + (-1.0720e-05)*phi + (0.015568))*pp + ((1.6135e-04)*phi*phi + (-3.7412e-04)*phi + (-0.051331));", "dp = dp + ((-2.6323e-09)*phi*phi + (1.0475e-05)*phi + (1.7346e-04))*pp*pp + ((3.4369e-05)*phi*phi + (-2.0506e-04)*phi + (-0.0033645))*pp + ((-3.1440e-04)*phi*phi + (8.5940e-04)*phi + (0.017928));", "dp = dp + ((-6.9452e-06)*phi*phi + (1.2777e-05)*phi + (0.0010526))*pp*pp + ((6.8395e-05)*phi*phi + (-1.7579e-04)*phi + (-0.014401))*pp + ((-5.8937e-05)*phi*phi + (5.0637e-04)*phi + (0.044588));", "dp = dp + ((8.6908e-06)*phi*phi + (-2.6211e-06)*phi + (-9.9693e-04))*pp*pp + ((-9.2982e-05)*phi*phi + (5.0393e-05)*phi + (0.01287))*pp + ((1.3405e-04)*phi*phi + (-1.5467e-04)*phi + (-0.036625));", "dp = dp + ((-1.8802e-06)*phi*phi +  (1.5345e-05)*phi +  (1.4538e-04))*pp*pp + ((-6.4792e-06)*phi*phi + (-1.8614e-04)*phi + (-7.4891e-04))*pp +  ((2.0656e-04)*phi*phi +  (4.4799e-04)*phi + (-0.0041372));", "dp = dp + ((-1.3704e-06)*phi*phi + (-2.2204e-05)*phi + (-8.5453e-06))*pp*pp + ((5.4827e-05)*phi*phi + (2.8622e-04)*phi + (-0.0010583))*pp + ((-3.7468e-04)*phi*phi + (-8.1873e-04)*phi + (0.0096448));", "dp = dp + ((-2.5598e-06)*phi*phi +  (2.5822e-05)*phi +  (3.6894e-04))*pp*pp + ((-4.1048e-06)*phi*phi + (-3.5369e-04)*phi + (-0.0038407))*pp + ((2.3498e-04)*phi*phi +   (0.0011169)*phi + (0.0054669));", "dp = dp +  ((1.5502e-05)*phi*phi + (-2.0620e-05)*phi +  (-0.0015768))*pp*pp + ((-1.8364e-04)*phi*phi +  (2.7179e-04)*phi +   (0.021456))*pp + ((4.0783e-04)*phi*phi + (-8.0724e-04)*phi + (-0.066293));"]
# eqs_list = ["dp =       ((7.3197e-07)*phi*phi +  (6.1298e-06)*phi +  (5.5676e-04))*pp*pp + ((-7.5385e-06)*phi*phi + (-3.2695e-05)*phi +  (-0.0078505))*pp +  ((3.7310e-05)*phi*phi + (-1.4014e-04)*phi +  (0.024597))", "dp = dp +  ((1.4970e-07)*phi*phi + (-3.5725e-05)*phi + (-6.9360e-04))*pp*pp + ((-1.1580e-07)*phi*phi +  (4.6423e-04)*phi +   (0.0088647))*pp + ((-1.9624e-05)*phi*phi +  (-0.0012901)*phi +  (-0.024413))", "dp = dp + ((-1.4108e-09)*phi*phi +  (3.9037e-09)*phi + (-5.0018e-08))*pp*pp +  ((3.3209e-08)*phi*phi + (-7.2877e-08)*phi + (-1.3088e-07))*pp + ((-1.0336e-07)*phi*phi +  (1.5223e-07)*phi + (3.2898e-06))"]


# # Sector 6
# eqs_list = ["dp = ((-1.7832e-06)*phi*phi + (1.6966e-05)*phi + (-0.0010273))*pp*pp + ((3.5802e-05)*phi*phi + (-7.7697e-05)*phi + (0.018163))*pp + ((-8.7108e-05)*phi*phi + (1.7289e-04)*phi + (-0.044444));", "dp = dp + ((1.1422e-05)*phi*phi + (1.6369e-05)*phi + (-7.5436e-04))*pp*pp + ((-1.5646e-04)*phi*phi + (-2.5678e-04)*phi + (0.010662))*pp + ((4.8595e-04)*phi*phi + (0.0010086)*phi + (-0.032133));", "dp = dp + ((-1.6348e-05)*phi*phi + (-1.0959e-06)*phi + (8.7809e-04))*pp*pp + ((2.3135e-04)*phi*phi + (1.1761e-04)*phi + (-0.012469))*pp + ((-7.3188e-04)*phi*phi + (-8.8192e-04)*phi + (0.039663));", "dp = dp + ((1.8650e-05)*phi*phi + (1.1757e-05)*phi + (-5.2514e-04))*pp*pp + ((-2.6206e-04)*phi*phi + (-2.6712e-04)*phi + (0.0074623))*pp + ((8.3426e-04)*phi*phi + (0.0013708)*phi + (-0.024749));", "dp = dp + ((-1.3151e-05)*phi*phi + (-5.9373e-05)*phi + (-4.5769e-04))*pp*pp +  ((1.8344e-04)*phi*phi +  (8.9377e-04)*phi +   (0.0067169))*pp + ((-5.7639e-04)*phi*phi +  (-0.0031197)*phi + (-0.022393));", "dp = dp + ((3.7315e-06)*phi*phi + (2.6829e-05)*phi + (8.5940e-04))*pp*pp + ((-4.8253e-05)*phi*phi + (-4.2904e-04)*phi + (-0.012406))*pp + ((1.2664e-04)*phi*phi + (0.0015771)*phi + (0.040887));", "dp = dp + ((-1.7688e-08)*phi*phi +  (1.5406e-05)*phi + (-3.6864e-04))*pp*pp + ((-6.1710e-06)*phi*phi + (-1.8641e-04)*phi +  (0.0052647))*pp + ((5.1462e-05)*phi*phi +  (4.8340e-04)*phi + (-0.016055));"]
# eqs_list = ["dp =       ((6.4400e-06)*phi*phi + (8.1945e-06)*phi + (-1.8831e-04))*pp*pp + ((-1.0280e-04)*phi*phi + (-1.2099e-04)*phi +  (0.0035858))*pp +  ((3.9912e-04)*phi*phi + (3.2813e-04)*phi + (-0.016191))", "dp = dp + ((-5.6636e-06)*phi*phi + (1.5947e-05)*phi + (1.8200e-04))*pp*pp + ((9.2222e-05)*phi*phi + (-1.9799e-04)*phi + (-0.0036102))*pp + ((-3.4872e-04)*phi*phi + (5.8949e-04)*phi + (0.016365))", "dp = dp + ((-3.8044e-06)*phi*phi + (3.8009e-05)*phi + (4.3000e-07))*pp*pp + ((5.1125e-05)*phi*phi + (-5.1083e-04)*phi + (-4.3000e-06))*pp + ((-1.6087e-04)*phi*phi + (0.0016078)*phi + (7.1800e-06))", "dp = dp + ((3.8006e-06)*phi*phi + (-3.8027e-05)*phi + (-2.5000e-07))*pp*pp + ((-5.1091e-05)*phi*phi + (5.1102e-04)*phi + (3.4700e-06))*pp + ((1.6079e-04)*phi*phi + (-0.0016082)*phi + (-8.4400e-06))"]

# for eq in eqs_list:
#     para_terms_ii = extract_parameters(eq)
#     for ii, pars in enumerate(para_terms_ii):
#         para_terms[ii] += pars
    

# print(para_terms)


# parameters = ["Aa", "Ab", "Ac", "Ba", "Bb", "Bc", "Ca", "Cb", "Cc"]
# eq_base    = "\ndp = dp + ((Aa)*phi*phi + (Ab)*phi + (Ac))*pp*pp + ((Ba)*phi*phi + (Bb)*phi + (Bc))*pp + ((Ca)*phi*phi + (Cb)*phi + (Cc));"

# for ii, para in enumerate(para_terms):
#     eq_base = eq_base.replace(parameters[ii], str(print_rounded_str(para, rounding=5)))
# #     print(eq_base, "\n")
    
# print("\nDone:", eq_base, "\n")


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[7.726e-07, 2.41235e-05, -6.129999999999985e-06, -1.0544000000000006e-05, -0.00031879, -2.5229999999999943e-05, 5.031999999999999e-05, 0.0009172200000000001, 0.00017274000000000055]

Done: 
dp = dp + ((7.7260e-07)*phi*phi + (2.4124e-05)*phi + (-6.1300e-06))*pp*pp + ((-1.0544e-05)*phi*phi + (-3.1879e-04)*phi + (-2.5230e-05))*pp + ((5.0320e-05)*phi*phi + (9.1722e-04)*phi + (1.7274e-04)); 



In [65]:
Cors_to_Combine = """
if(sec == 1){
    dp =      ((-5.5619e-07)*phi*phi +  (1.2176e-05)*phi +   (0.0019766))*pp*pp +  ((2.2974e-06)*phi*phi + (-3.2344e-04)*phi +  (-0.018279))*pp + ((-3.3287e-06)*phi*phi +  (1.2371e-04)*phi + (0.037557));
    dp = dp + ((-1.1071e-07)*phi*phi +  (1.1611e-05)*phi + (-6.8815e-04))*pp*pp + ((-2.2037e-06)*phi*phi + (-1.8548e-04)*phi +  (0.0055018))*pp +  ((2.3393e-07)*phi*phi +  (1.2655e-04)*phi + (-0.0055127));
    dp = dp + ((-1.0665e-06)*phi*phi + (-9.2758e-06)*phi +  (2.1876e-04))*pp*pp +  ((6.5297e-06)*phi*phi +  (2.7729e-05)*phi + (-0.0010923))*pp + ((-1.4148e-07)*phi*phi +  (2.9491e-05)*phi + (-0.0043183));
}
if(sec == 2){
    dp =      ((-3.0769e-06)*phi*phi +  (5.2387e-06)*phi +   (0.0025674))*pp*pp +  ((1.8124e-05)*phi*phi + (-1.7621e-04)*phi +  (-0.020154))*pp + ((-1.2558e-05)*phi*phi +  (1.2769e-04)*phi + (0.033566));
    dp = dp + ((-1.2998e-06)*phi*phi + (-2.7296e-05)*phi + (-1.2792e-04))*pp*pp +  ((6.8931e-06)*phi*phi +  (1.6941e-04)*phi +  (0.0013802))*pp + ((-8.9689e-06)*phi*phi + (-2.5644e-04)*phi + (-0.0032964));
    dp = dp + ((-9.8764e-08)*phi*phi + (-1.9100e-05)*phi + (-3.9391e-04))*pp*pp + ((-3.3432e-07)*phi*phi +  (1.0039e-04)*phi +  (0.0033748))*pp +  ((5.3722e-06)*phi*phi + (-1.1331e-04)*phi + (-0.0070953));
}
if(sec == 3){
    dp =       ((1.3125e-06)*phi*phi + (-5.9180e-05)*phi +   (0.0014564))*pp*pp + ((-7.2451e-06)*phi*phi +  (6.3174e-04)*phi +  (-0.017146))*pp +  ((1.0599e-05)*phi*phi + (-8.1889e-04)*phi + (0.029497));
    dp = dp + ((-1.7131e-06)*phi*phi +  (5.7922e-05)*phi +  (5.9580e-04))*pp*pp +  ((1.1049e-05)*phi*phi + (-2.7280e-04)*phi + (-0.0039213))*pp + ((-8.5944e-06)*phi*phi +  (2.8071e-04)*phi + (-1.3545e-04));
    dp = dp + ((-4.0314e-07)*phi*phi +  (4.1308e-06)*phi + (-5.3057e-04))*pp*pp +  ((1.3308e-06)*phi*phi +  (1.2769e-05)*phi +  (0.0044938))*pp +  ((2.0059e-06)*phi*phi +  (9.1931e-06)*phi + (-0.0076076));
}
if(sec == 4){
    dp =      ((-3.3265e-07)*phi*phi + (-3.0209e-05)*phi +    (0.001713))*pp*pp +  ((7.3058e-06)*phi*phi +  (3.1596e-04)*phi +  (-0.017057))*pp + ((-1.3587e-05)*phi*phi + (-2.3543e-04)*phi + (0.034361));
    dp = dp +  ((9.2471e-07)*phi*phi +  (7.2685e-06)*phi + (-1.4055e-05))*pp*pp + ((-8.2856e-06)*phi*phi +  (1.0956e-05)*phi +  (0.0013298))*pp +  ((1.1312e-05)*phi*phi + (-3.6642e-05)*phi + (-0.004602));
    dp = dp + ((-1.2205e-06)*phi*phi +  (1.4853e-05)*phi +  (2.6654e-04))*pp*pp +  ((8.0883e-06)*phi*phi + (-8.1258e-05)*phi + (-0.0017054))*pp + ((-6.7779e-06)*phi*phi + (-4.1304e-05)*phi + (0.0013179));
    dp = dp +  ((2.4054e-07)*phi*phi + (-3.9757e-05)*phi +  (3.5924e-04))*pp*pp + ((-3.0307e-07)*phi*phi +  (3.2370e-04)*phi + (-0.0024836))*pp + ((-3.9735e-06)*phi*phi + (-2.7823e-04)*phi + (9.4398e-04));
}
if(sec == 5){
    dp =       ((4.9522e-08)*phi*phi +  (1.6615e-06)*phi +   (0.0013238))*pp*pp + ((-7.7824e-07)*phi*phi + (-1.8424e-05)*phi +  (-0.013754))*pp +  ((6.2245e-06)*phi*phi +  (5.4907e-05)*phi + (0.027401));
    dp = dp +  ((8.9196e-08)*phi*phi + (-3.7341e-05)*phi +  (1.5161e-05))*pp*pp +  ((3.2196e-06)*phi*phi +  (3.1272e-04)*phi + (-0.0012442))*pp + ((-9.1313e-06)*phi*phi + (-5.1308e-04)*phi + (0.0042737));
    dp = dp +  ((2.0598e-06)*phi*phi +  (2.0426e-05)*phi + (-1.5120e-04))*pp*pp + ((-1.2444e-05)*phi*phi + (-1.3032e-04)*phi + (7.0147e-04))*pp +  ((1.2403e-05)*phi*phi +  (7.1482e-05)*phi + (-0.0013052));
}
if(sec == 6){
    dp =      ((-1.1641e-06)*phi*phi +  (3.6397e-05)*phi +   (0.0017669))*pp*pp +  ((1.2615e-05)*phi*phi + (-3.1012e-04)*phi +  (-0.018168))*pp + ((-1.9665e-05)*phi*phi +  (6.8981e-05)*phi + (0.038744));
    dp = dp + ((-4.7604e-08)*phi*phi + (-8.9182e-06)*phi + (-9.4440e-05))*pp*pp + ((-1.9801e-06)*phi*phi +  (2.6388e-05)*phi +  (0.0014914))*pp +  ((5.5219e-06)*phi*phi + (-9.8614e-05)*phi + (-0.0038784));
    dp = dp +  ((8.1876e-07)*phi*phi + (-1.2894e-05)*phi + (-4.6578e-04))*pp*pp + ((-6.8450e-06)*phi*phi +  (8.5513e-05)*phi +  (0.0035454))*pp +  ((1.0147e-05)*phi*phi + (-1.0325e-04)*phi + (-0.0054159));
}
"""

for sec in range(1, 7, 1):
    Cors_to_Combine = Cors_to_Combine.replace("".join(["if(sec == ", str(sec), "){\n"]), "".join(["\n# Sector ", str(sec), ":\neqs_sec_", str(sec), " = [\""]))
    
Cors_to_Combine = Cors_to_Combine.replace("}\n", "\"]\n")
Cors_to_Combine = Cors_to_Combine.replace(";\n", "\", \"")
Cors_to_Combine = Cors_to_Combine.replace("    dp =", "dp =")
Cors_to_Combine = Cors_to_Combine.replace(", \"\"]", "]")

    
# Cors_to_Combine
print("\n\n", Cors_to_Combine, "\n\n")



 

# Sector 1:
eqs_sec_1 = ["dp =      ((-5.5619e-07)*phi*phi +  (1.2176e-05)*phi +   (0.0019766))*pp*pp +  ((2.2974e-06)*phi*phi + (-3.2344e-04)*phi +  (-0.018279))*pp + ((-3.3287e-06)*phi*phi +  (1.2371e-04)*phi + (0.037557))", "dp = dp + ((-1.1071e-07)*phi*phi +  (1.1611e-05)*phi + (-6.8815e-04))*pp*pp + ((-2.2037e-06)*phi*phi + (-1.8548e-04)*phi +  (0.0055018))*pp +  ((2.3393e-07)*phi*phi +  (1.2655e-04)*phi + (-0.0055127))", "dp = dp + ((-1.0665e-06)*phi*phi + (-9.2758e-06)*phi +  (2.1876e-04))*pp*pp +  ((6.5297e-06)*phi*phi +  (2.7729e-05)*phi + (-0.0010923))*pp + ((-1.4148e-07)*phi*phi +  (2.9491e-05)*phi + (-0.0043183))"]

# Sector 2:
eqs_sec_2 = ["dp =      ((-3.0769e-06)*phi*phi +  (5.2387e-06)*phi +   (0.0025674))*pp*pp +  ((1.8124e-05)*phi*phi + (-1.7621e-04)*phi +  (-0.020154))*pp + ((-1.2558e-05)*phi*phi +  (1.2769e-04)*phi + (0.033566))", "dp = dp + ((-1.2998e-06)*phi*phi + (-2.7296e-05)*phi + (-1.2792e-04))*pp*pp +  ((6.8931e-06)*phi*phi +  (1.6941e-04)*phi +  (0.0013

In [66]:
import re

def extract_parameters(eq):
    # Define a regular expression pattern to match the parameter values
    pattern = r"[-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?"
    # Use the pattern to find all the parameter values in the equation
    params = re.findall(pattern, eq)
    # Convert the parameter values to floats and return them
    return [float(p) for p in params]


equation = "dp = dp + ((0)*phi*phi + (0)*phi + (0))*pp*pp + ((0)*phi*phi + (0)*phi + (0))*pp + ((0)*phi*phi + (0)*phi + (0));"


# Sector 1:
eqs_sec_1 = ["dp =      ((-5.5619e-07)*phi*phi +  (1.2176e-05)*phi +   (0.0019766))*pp*pp +  ((2.2974e-06)*phi*phi + (-3.2344e-04)*phi +  (-0.018279))*pp + ((-3.3287e-06)*phi*phi +  (1.2371e-04)*phi + (0.037557))", "dp = dp + ((-1.1071e-07)*phi*phi +  (1.1611e-05)*phi + (-6.8815e-04))*pp*pp + ((-2.2037e-06)*phi*phi + (-1.8548e-04)*phi +  (0.0055018))*pp +  ((2.3393e-07)*phi*phi +  (1.2655e-04)*phi + (-0.0055127))", "dp = dp + ((-1.0665e-06)*phi*phi + (-9.2758e-06)*phi +  (2.1876e-04))*pp*pp +  ((6.5297e-06)*phi*phi +  (2.7729e-05)*phi + (-0.0010923))*pp + ((-1.4148e-07)*phi*phi +  (2.9491e-05)*phi + (-0.0043183))"]

# Sector 2:
eqs_sec_2 = ["dp =      ((-3.0769e-06)*phi*phi +  (5.2387e-06)*phi +   (0.0025674))*pp*pp +  ((1.8124e-05)*phi*phi + (-1.7621e-04)*phi +  (-0.020154))*pp + ((-1.2558e-05)*phi*phi +  (1.2769e-04)*phi + (0.033566))", "dp = dp + ((-1.2998e-06)*phi*phi + (-2.7296e-05)*phi + (-1.2792e-04))*pp*pp +  ((6.8931e-06)*phi*phi +  (1.6941e-04)*phi +  (0.0013802))*pp + ((-8.9689e-06)*phi*phi + (-2.5644e-04)*phi + (-0.0032964))", "dp = dp + ((-9.8764e-08)*phi*phi + (-1.9100e-05)*phi + (-3.9391e-04))*pp*pp + ((-3.3432e-07)*phi*phi +  (1.0039e-04)*phi +  (0.0033748))*pp +  ((5.3722e-06)*phi*phi + (-1.1331e-04)*phi + (-0.0070953))"]

# Sector 3:
eqs_sec_3 = ["dp =       ((1.3125e-06)*phi*phi + (-5.9180e-05)*phi +   (0.0014564))*pp*pp + ((-7.2451e-06)*phi*phi +  (6.3174e-04)*phi +  (-0.017146))*pp +  ((1.0599e-05)*phi*phi + (-8.1889e-04)*phi + (0.029497))", "dp = dp + ((-1.7131e-06)*phi*phi +  (5.7922e-05)*phi +  (5.9580e-04))*pp*pp +  ((1.1049e-05)*phi*phi + (-2.7280e-04)*phi + (-0.0039213))*pp + ((-8.5944e-06)*phi*phi +  (2.8071e-04)*phi + (-1.3545e-04))", "dp = dp + ((-4.0314e-07)*phi*phi +  (4.1308e-06)*phi + (-5.3057e-04))*pp*pp +  ((1.3308e-06)*phi*phi +  (1.2769e-05)*phi +  (0.0044938))*pp +  ((2.0059e-06)*phi*phi +  (9.1931e-06)*phi + (-0.0076076))"]

# Sector 4:
eqs_sec_4 = ["dp =      ((-3.3265e-07)*phi*phi + (-3.0209e-05)*phi +    (0.001713))*pp*pp +  ((7.3058e-06)*phi*phi +  (3.1596e-04)*phi +  (-0.017057))*pp + ((-1.3587e-05)*phi*phi + (-2.3543e-04)*phi + (0.034361))", "dp = dp +  ((9.2471e-07)*phi*phi +  (7.2685e-06)*phi + (-1.4055e-05))*pp*pp + ((-8.2856e-06)*phi*phi +  (1.0956e-05)*phi +  (0.0013298))*pp +  ((1.1312e-05)*phi*phi + (-3.6642e-05)*phi + (-0.004602))", "dp = dp + ((-1.2205e-06)*phi*phi +  (1.4853e-05)*phi +  (2.6654e-04))*pp*pp +  ((8.0883e-06)*phi*phi + (-8.1258e-05)*phi + (-0.0017054))*pp + ((-6.7779e-06)*phi*phi + (-4.1304e-05)*phi + (0.0013179))", "dp = dp +  ((2.4054e-07)*phi*phi + (-3.9757e-05)*phi +  (3.5924e-04))*pp*pp + ((-3.0307e-07)*phi*phi +  (3.2370e-04)*phi + (-0.0024836))*pp + ((-3.9735e-06)*phi*phi + (-2.7823e-04)*phi + (9.4398e-04))"]

# Sector 5:
eqs_sec_5 = ["dp =       ((4.9522e-08)*phi*phi +  (1.6615e-06)*phi +   (0.0013238))*pp*pp + ((-7.7824e-07)*phi*phi + (-1.8424e-05)*phi +  (-0.013754))*pp +  ((6.2245e-06)*phi*phi +  (5.4907e-05)*phi + (0.027401))", "dp = dp +  ((8.9196e-08)*phi*phi + (-3.7341e-05)*phi +  (1.5161e-05))*pp*pp +  ((3.2196e-06)*phi*phi +  (3.1272e-04)*phi + (-0.0012442))*pp + ((-9.1313e-06)*phi*phi + (-5.1308e-04)*phi + (0.0042737))", "dp = dp +  ((2.0598e-06)*phi*phi +  (2.0426e-05)*phi + (-1.5120e-04))*pp*pp + ((-1.2444e-05)*phi*phi + (-1.3032e-04)*phi + (7.0147e-04))*pp +  ((1.2403e-05)*phi*phi +  (7.1482e-05)*phi + (-0.0013052))"]

# Sector 6:
eqs_sec_6 = ["dp =      ((-1.1641e-06)*phi*phi +  (3.6397e-05)*phi +   (0.0017669))*pp*pp +  ((1.2615e-05)*phi*phi + (-3.1012e-04)*phi +  (-0.018168))*pp + ((-1.9665e-05)*phi*phi +  (6.8981e-05)*phi + (0.038744))", "dp = dp + ((-4.7604e-08)*phi*phi + (-8.9182e-06)*phi + (-9.4440e-05))*pp*pp + ((-1.9801e-06)*phi*phi +  (2.6388e-05)*phi +  (0.0014914))*pp +  ((5.5219e-06)*phi*phi + (-9.8614e-05)*phi + (-0.0038784))", "dp = dp +  ((8.1876e-07)*phi*phi + (-1.2894e-05)*phi + (-4.6578e-04))*pp*pp + ((-6.8450e-06)*phi*phi +  (8.5513e-05)*phi +  (0.0035454))*pp +  ((1.0147e-05)*phi*phi + (-1.0325e-04)*phi + (-0.0054159))"]




eqs_list = [["sec == 1", eqs_sec_1], ["sec == 2", eqs_sec_2], ["sec == 3", eqs_sec_3], ["sec == 4", eqs_sec_4], ["sec == 5", eqs_sec_5], ["sec == 6", eqs_sec_6]]

# eqs_list = [["sec == 1", eqs_sec_1]]


para_terms = {}
parameters = ["Aa", "Ab", "Ac", "Ba", "Bb", "Bc", "Ca", "Cb", "Cc"]
for eqs in eqs_list:
    para_terms[eqs[0]] = extract_parameters(equation)
    # print(extract_parameters(equation))
    for eq in eqs[1]:
        para_terms_ii = extract_parameters(eq)
        for ii, pars in enumerate(para_terms_ii):
            para_terms[eqs[0]][ii] += pars
    # print(para_terms, "\n")
    eq_base    = "dp = ((Aa)*phi*phi + (Ab)*phi + (Ac))*pp*pp + ((Ba)*phi*phi + (Bb)*phi + (Bc))*pp + ((Ca)*phi*phi + (Cb)*phi + (Cc));"
    for ii, para in enumerate(para_terms[eqs[0]]):
        eq_base = eq_base.replace(parameters[ii], str(print_rounded_str(para, rounding=10)))
    for ii in range(0, 5, 1):
        eq_fin = eq_base.replace("0e", "e")
        if(str(eq_fin) == str(eq_base)):
            # print("test")
            break
        else:
            eq_base = eq_fin
    print("".join(["if(", eqs[0], "){\n    ", str(eq_fin), "\n}"]))
    
# print("\nDone:", eq_base, "\n")


if(sec == 1){
    dp = ((-1.7334e-06)*phi*phi + (1.45112e-05)*phi + (0.00150721))*pp*pp + ((6.6234e-06)*phi*phi + (-4.81191e-04)*phi + (-0.0138695))*pp + ((-3.23625e-06)*phi*phi + (2.79751e-04)*phi + (0.027726));
}
if(sec == 2){
    dp = ((-4.475464e-06)*phi*phi + (-4.11573e-05)*phi + (0.00204557))*pp*pp + ((2.468278e-05)*phi*phi + (9.3590e-05)*phi + (-0.015399))*pp + ((-1.61547e-05)*phi*phi + (-2.4206e-04)*phi + (0.0231743));
}
if(sec == 3){
    dp = ((-8.0374e-07)*phi*phi + (2.8728e-06)*phi + (0.00152163))*pp*pp + ((5.1347e-06)*phi*phi + (3.71709e-04)*phi + (-0.0165735))*pp + ((4.0105e-06)*phi*phi + (-5.289869e-04)*phi + (0.02175395));
}
if(sec == 4){
    dp = ((-3.8790e-07)*phi*phi + (-4.78445e-05)*phi + (0.002324725))*pp*pp + ((6.80543e-06)*phi*phi + (5.69358e-04)*phi + (-0.0199162))*pp + ((-1.30264e-05)*phi*phi + (-5.91606e-04)*phi + (0.03202088));
}
if(sec == 5){
    dp = ((2.198518e-06)*phi*phi + (-1.52535e-05)*phi + (0.001187761))*pp*pp + ((-1.000264e-05)*phi*phi + (1.63976e-04

In [59]:
print(print_rounded_str((2.503612e-06) + (-2.5661e-06), rounding=5))

-6.2488e-08


In [1]:
Output = """
For  Sector 1 -- reg1  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.9307
0.936
0.93871
0.9402
0.94043
0.9425
0.9425
0.9425
0.9425
0.94107
0.94007
0.93983
0.9398
0.941
0.9425

For  Sector 1 -- reg2  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.95013
0.946
0.94452
0.9425
0.9425
0.94486
0.9425
0.9425
0.9419
0.94133
0.94087
0.94111
0.94119
0.9416
0.9425

For  Sector 1 -- reg3  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.964
0.95
0.95356
0.9495
0.9495
0.94829
0.94799
0.94612
0.94553
0.9425
0.9425
0.94202
0.94173
0.94188
0.9425

For  Sector 2 -- reg1  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.93763
0.9411
0.94196
0.94091
0.9425
0.94132
0.9412
0.94169
0.94054
0.94166
0.9413
0.94067
0.94095
0.94184
0.9425

For  Sector 2 -- reg2  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.9442
0.92822
0.92772
0.93404
0.9355
0.9355
0.93973
0.94107
0.9425
0.9425
0.9425
0.9425
0.9425
0.94245
0.9425

For  Sector 2 -- reg3  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.92865
0.93206
0.932
0.93513
0.9355
0.93806
0.93959
0.93943
0.94051
0.94128
0.94244
0.94231
0.94217
0.94193
0.9425

For  Sector 3 -- reg1  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.94391
0.94312
0.94476
0.9355
0.9425
0.94168
0.93903
0.94099
0.94055
0.93999
0.93984
0.93991
0.94066
0.94198
0.94222

For  Sector 3 -- reg2  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.936
0.93778
0.93906
0.9355
0.93709
0.9395
0.94054
0.94195
0.9416
0.94239
0.94237
0.94199
0.94248
0.94213
0.94168

For  Sector 3 -- reg3  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.92918
0.932
0.936
0.93897
0.93933
0.94171
0.94191
0.94434
0.94439
0.9425
0.94505
0.94456
0.9425
0.9425
0.9425

For  Sector 4 -- reg1  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.93866
0.94568
0.94112
0.94044
0.94152
0.93997
0.93934
0.9407
0.93951
0.93942
0.93911
0.93914
0.93998
0.94012
0.941

For  Sector 4 -- reg2  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.9385
0.93702
0.94119
0.9355
0.9355
0.9422
0.9425
0.94205
0.9424
0.9425
0.9425
0.9425
0.9425
0.9425
0.9425

For  Sector 4 -- reg3  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.94196
0.932
0.94091
0.94166
0.9355
0.93948
0.94014
0.94115
0.94142
0.94166
0.94166
0.94188
0.94217
0.94171
0.94091

For  Sector 5 -- reg1  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.94518
0.9428
0.94004
0.94076
0.94231
0.9425
0.94189
0.9425
0.94218
0.9419
0.94191
0.94145
0.94116
0.94013
0.9355

For  Sector 5 -- reg2  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.95
0.9498
0.94534
0.9425
0.94085
0.94048
0.94202
0.94122
0.94207
0.94219
0.9425
0.9425
0.9425
0.94097
0.9355

For  Sector 5 -- reg3  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.93973
0.93985
0.93843
0.94036
0.93793
0.94006
0.94131
0.9419
0.94405
0.9449
0.94629
0.94578
0.94543
0.94235
0.9355

For  Sector 6 -- reg1  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.946
0.94078
0.936
0.93892
0.93868
0.94005
0.94194
0.94224
0.94232
0.9425
0.94243
0.94242
0.9425
0.94223
0.9355

For  Sector 6 -- reg2  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.946
0.94369
0.93715
0.93737
0.93905
0.9425
0.94222
0.9425
0.94432
0.94627
0.94641
0.94702
0.94729
0.9425
0.93476

For  Sector 6 -- reg3  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.95
0.946
0.94468
0.9355
0.9425
0.94239
0.94291
0.9425
0.9425
0.9425
0.9425
0.9425
0.9425
0.9421
0.9411]"""



Output = """
For  Sector 1 -- reg1  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.9307
0.93728
0.93913
0.94025
0.93912
0.9425
0.9403
0.93998
0.94002
0.9389
0.93854
0.9392
0.94041
0.9425
0.94842

For  Sector 1 -- reg2  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.92633
0.9267
0.92871
0.93269
0.9355
0.94039
0.94098
0.94149
0.94112
0.94065
0.94002
0.93987
0.93945
0.93943
0.94246

For  Sector 1 -- reg3  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.918
0.922
0.927
0.93184
0.93534
0.9355
0.93943
0.93956
0.94058
0.94002
0.93992
0.93963
0.93983
0.94073
0.9425

For  Sector 2 -- reg1  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.94228
0.94313
0.94404
0.94236
0.9425
0.94181
0.94143
0.94185
0.94076
0.9421
0.94217
0.94214
0.9425
0.9425
0.9425

For  Sector 2 -- reg2  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.9442
0.9293
0.9384
0.94238
0.9425
0.94129
0.94209
0.94219
0.9425
0.9425
0.9425
0.9425
0.9425
0.9425
0.9425

For  Sector 2 -- reg3  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.9303
0.94551
0.946
0.9425
0.94365
0.9425
0.9425
0.94147
0.94175
0.942
0.9425
0.9425
0.9425
0.9425
0.94584

For  Sector 3 -- reg1  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.94391
0.94312
0.94476
0.9355
0.9425
0.94168
0.93903
0.94099
0.94055
0.93999
0.93984
0.93991
0.94066
0.94198
0.94222

For  Sector 3 -- reg2  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.936
0.93778
0.93906
0.9355
0.93709
0.9395
0.94054
0.94195
0.9416
0.94239
0.94237
0.94199
0.94248
0.94213
0.94168

For  Sector 3 -- reg3  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.92918
0.932
0.936
0.93897
0.93933
0.94171
0.94191
0.94434
0.94439
0.9425
0.94505
0.94456
0.9425
0.9425
0.9425

For  Sector 4 -- reg1  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.93866
0.94568
0.94112
0.94044
0.94152
0.93997
0.93934
0.9407
0.93951
0.93942
0.93911
0.93914
0.93998
0.94012
0.941

For  Sector 4 -- reg2  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.9385
0.93702
0.94119
0.9355
0.9355
0.9422
0.9425
0.94205
0.9424
0.9425
0.9425
0.9425
0.9425
0.9425
0.9425

For  Sector 4 -- reg3  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.94196
0.932
0.94091
0.94166
0.9355
0.93948
0.94014
0.94115
0.94142
0.94166
0.94166
0.94188
0.94217
0.94171
0.94091

For  Sector 5 -- reg1  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.94518
0.9428
0.94004
0.94076
0.94231
0.9425
0.94189
0.9425
0.94218
0.9419
0.94191
0.94145
0.94116
0.94013
0.9355

For  Sector 5 -- reg2  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.95
0.9498
0.94534
0.9425
0.94085
0.94048
0.94202
0.94122
0.94207
0.94219
0.9425
0.9425
0.9425
0.94097
0.9355

For  Sector 5 -- reg3  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.93973
0.93985
0.93843
0.94036
0.93793
0.94006
0.94131
0.9419
0.94405
0.9449
0.94629
0.94578
0.94543
0.94235
0.9355

For  Sector 6 -- reg1  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.946
0.94078
0.936
0.93892
0.93868
0.94005
0.94194
0.94224
0.94232
0.9425
0.94243
0.94242
0.9425
0.94223
0.9355

For  Sector 6 -- reg2  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.946
0.94369
0.93715
0.93737
0.93905
0.9425
0.94222
0.9425
0.94432
0.94627
0.94641
0.94702
0.94729
0.9425
0.93476

For  Sector 6 -- reg3  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:
0.95
0.946
0.94468
0.9355
0.9425
0.94239
0.94291
0.9425
0.9425
0.9425
0.9425
0.9425
0.9425
0.9421
0.9411]"""

# print(Output)
Output = Output.replace("Histo", "")
Output = str(Output.replace("For  Sector ", "MM_list['Sec_")).replace(" -- ", "'][")
Output = str((Output.replace("reg1", "0]   =")).replace("reg2", "-10] =")).replace("reg3", "10]  =")
Output = str(Output.replace("  the Correction:  mmEF_PipMMEF  the Missing Mass vs  el  peaks are:\n", " ["))
Output = str(Output.replace("  the Correction:  mmEF_PipMMEF  the Missing Mass vs  pip  peaks are:\n", " ["))
Output = str(Output.replace("  the Correction:  mmEF  the Missing Mass vs  el  peaks are:\n", " ["))
Output = Output.replace("\n\n", "]")
Output = Output.replace("\n", ", ")
Output = Output.replace(", MM_list", "\nMM_list")
Output = Output.replace("MM_list", "\nMM_list")
print(Output)



MM_list['Sec_1'][0]   = [0.9307, 0.93728, 0.93913, 0.94025, 0.93912, 0.9425, 0.9403, 0.93998, 0.94002, 0.9389, 0.93854, 0.9392, 0.94041, 0.9425, 0.94842]
MM_list['Sec_1'][-10] = [0.92633, 0.9267, 0.92871, 0.93269, 0.9355, 0.94039, 0.94098, 0.94149, 0.94112, 0.94065, 0.94002, 0.93987, 0.93945, 0.93943, 0.94246]
MM_list['Sec_1'][10]  = [0.918, 0.922, 0.927, 0.93184, 0.93534, 0.9355, 0.93943, 0.93956, 0.94058, 0.94002, 0.93992, 0.93963, 0.93983, 0.94073, 0.9425]
MM_list['Sec_2'][0]   = [0.94228, 0.94313, 0.94404, 0.94236, 0.9425, 0.94181, 0.94143, 0.94185, 0.94076, 0.9421, 0.94217, 0.94214, 0.9425, 0.9425, 0.9425]
MM_list['Sec_2'][-10] = [0.9442, 0.9293, 0.9384, 0.94238, 0.9425, 0.94129, 0.94209, 0.94219, 0.9425, 0.9425, 0.9425, 0.9425, 0.9425, 0.9425, 0.9425]
MM_list['Sec_2'][10]  = [0.9303, 0.94551, 0.946, 0.9425, 0.94365, 0.9425, 0.9425, 0.94147, 0.94175, 0.942, 0.9425, 0.9425, 0.9425, 0.9425, 0.94584]
MM_list['Sec_3'][0]   = [0.94391, 0.94312, 0.94476, 0.9355, 0.9425, 0.94168, 0.939

In [2]:
Output = """
if(sec == 1){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 1] is:
    dp = dp + ((1.3267e-05)*phi*phi + (-3.8513e-05)*phi + (-6.4671e-04))*pp*pp + ((-1.9408e-04)*phi*phi + (4.0799e-04)*phi + (0.008038))*pp + ((7.1060e-04)*phi*phi + (-6.7122e-04)*phi + (-0.024983));
}
if(sec == 2){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 2] is:
    dp = dp + ((-4.0296e-06)*phi*phi + (2.4382e-05)*phi + (-3.5219e-04))*pp*pp + ((6.6680e-05)*phi*phi + (-3.4092e-04)*phi + (0.0042897))*pp + ((-2.4093e-04)*phi*phi + (0.0010581)*phi + (-0.015263));
}
if(sec == 3){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 3] is:
    dp = dp + ((-4.8001e-06)*phi*phi + (-5.6033e-05)*phi + (-2.2236e-04))*pp*pp + ((7.2496e-05)*phi*phi + (7.0810e-04)*phi + (0.0021065))*pp + ((-2.0874e-04)*phi*phi + (-0.0017164)*phi + (-0.0077417));
}
if(sec == 4){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 4] is:
    dp = dp + ((-6.3519e-06)*phi*phi + (3.0428e-05)*phi + (6.2350e-04))*pp*pp + ((9.8415e-05)*phi*phi + (-4.8133e-04)*phi + (-0.009781))*pp + ((-3.2711e-04)*phi*phi + (0.0016966)*phi + (0.031834));
}
if(sec == 5){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 5] is:
    dp = dp + ((1.1825e-06)*phi*phi + (1.1477e-05)*phi + (3.3236e-04))*pp*pp + ((-1.2024e-05)*phi*phi + (-8.6847e-05)*phi + (-0.0057913))*pp + ((4.7082e-05)*phi*phi + (-2.1875e-05)*phi + (0.020465));
}
if(sec == 6){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 6] is:
    dp = dp + ((8.8433e-06)*phi*phi + (6.0471e-06)*phi + (-6.2909e-04))*pp*pp + ((-1.2732e-04)*phi*phi + (-9.3674e-05)*phi + (0.0078974))*pp + ((4.5299e-04)*phi*phi + (2.5762e-04)*phi + (-0.025347));
}
"""
Output = """
if(sec == 1){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Without Pion)][Sector 1] is:
    dp = dp + ((-1.0625e-07)*phi*phi + (-1.4010e-05)*phi + (-5.8009e-04))*pp*pp + ((2.2990e-05)*phi*phi + (2.2025e-04)*phi + (0.0088427))*pp + ((-1.8280e-04)*phi*phi + (-7.9973e-04)*phi + (-0.031216));
}
if(sec == 2){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Without Pion)][Sector 2] is:
    dp = dp + ((1.2638e-05)*phi*phi + (5.1213e-05)*phi + (-0.0010273))*pp*pp + ((-1.6358e-04)*phi*phi + (-6.9213e-04)*phi + (0.012737))*pp + ((4.7856e-04)*phi*phi + (0.0021964)*phi + (-0.034427));
}
if(sec == 3){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Without Pion)][Sector 3] is:
    dp = dp + ((7.8585e-06)*phi*phi + (3.6004e-05)*phi + (-9.3161e-04))*pp*pp + ((-1.0079e-04)*phi*phi + (-4.7046e-04)*phi + (0.011898))*pp + ((2.7810e-04)*phi*phi + (0.0013496)*phi + (-0.033007));
}
if(sec == 4){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Without Pion)][Sector 4] is:
    dp = dp + ((-1.4941e-05)*phi*phi + (1.0707e-05)*phi + (0.0012096))*pp*pp + ((1.8141e-04)*phi*phi + (-2.1963e-04)*phi + (-0.015432))*pp + ((-4.6408e-04)*phi*phi + (0.0011872)*phi + (0.04357));
}
if(sec == 5){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Without Pion)][Sector 5] is:
    dp = dp + ((-2.0286e-05)*phi*phi + (-1.0690e-05)*phi + (0.0011941))*pp*pp + ((2.6340e-04)*phi*phi + (1.9392e-04)*phi + (-0.016763))*pp + ((-7.3645e-04)*phi*phi + (-8.0943e-04)*phi + (0.05283));
}
if(sec == 6){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Without Pion)][Sector 6] is:
    dp = dp + ((4.6312e-06)*phi*phi + (-5.2430e-06)*phi + (-6.0825e-04))*pp*pp + ((-5.5971e-05)*phi*phi + (7.8651e-05)*phi + (0.0077788))*pp + ((1.4252e-04)*phi*phi + (-2.5100e-04)*phi + (-0.024202));
}
"""


Output = """
if(sec == 1){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 1] is:
    dp = dp + ((1.3320e-05)*phi*phi + (-1.7968e-05)*phi + (-7.1449e-04))*pp*pp + ((-1.8680e-04)*phi*phi + (1.3628e-04)*phi + (0.0090621))*pp + ((6.5236e-04)*phi*phi + (1.1485e-04)*phi + (-0.025553));
}
if(sec == 2){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 2] is:
    dp = dp + ((-4.0056e-06)*phi*phi + (2.4598e-05)*phi + (-3.5394e-04))*pp*pp + ((6.6444e-05)*phi*phi + (-3.4250e-04)*phi + (0.004308))*pp + ((-2.4019e-04)*phi*phi + (0.0010546)*phi + (-0.015295));
}
if(sec == 3){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 3] is:
    dp = dp + ((-4.7871e-06)*phi*phi + (-5.6129e-05)*phi + (-2.2466e-04))*pp*pp + ((7.2395e-05)*phi*phi + (7.0933e-04)*phi + (0.0021319))*pp + ((-2.0868e-04)*phi*phi + (-0.0017197)*phi + (-0.007789));
}
if(sec == 4){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 4] is:
    dp = dp + ((-6.4659e-06)*phi*phi + (3.4630e-05)*phi + (6.2207e-04))*pp*pp + ((9.9398e-05)*phi*phi + (-5.3345e-04)*phi + (-0.0097637))*pp + ((-3.2634e-04)*phi*phi + (0.0018233)*phi + (0.031806));
}
if(sec == 5){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 5] is:
    dp = dp + ((2.4359e-06)*phi*phi + (8.2700e-06)*phi + (2.2981e-04))*pp*pp + ((-2.7862e-05)*phi*phi + (-3.2455e-05)*phi + (-0.0046421))*pp + ((8.8557e-05)*phi*phi + (-2.4495e-04)*phi + (0.018327));
}
if(sec == 6){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 6] is:
    dp = dp + ((1.2945e-05)*phi*phi + (-1.2685e-06)*phi + (-0.0011454))*pp*pp + ((-1.7419e-04)*phi*phi + (1.7902e-05)*phi + (0.014147))*pp + ((5.5157e-04)*phi*phi + (-1.2575e-05)*phi + (-0.039022));
}
"""


Output = """
if(sec == 1){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 1] is:
    dp = dp + ((1.5083e-05)*phi*phi + (-1.6580e-06)*phi + (-7.9929e-04))*pp*pp + ((-2.1076e-04)*phi*phi + (-8.7896e-05)*phi + (0.010204))*pp + ((7.2743e-04)*phi*phi + (8.2690e-04)*phi + (-0.029093));
}
if(sec == 2){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 2] is:
    dp = dp + ((-4.3961e-06)*phi*phi + (3.7101e-05)*phi + (-1.8985e-04))*pp*pp + ((7.1683e-05)*phi*phi + (-5.1441e-04)*phi + (0.002065))*pp + ((-2.5634e-04)*phi*phi + (0.0016011)*phi + (-0.0082156));
}
if(sec == 3){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 3] is:
    dp = dp + ((-1.2403e-05)*phi*phi + (-5.6129e-05)*phi + (5.3692e-04))*pp*pp + ((1.7689e-04)*phi*phi + (7.0933e-04)*phi + (-0.0083175))*pp + ((-5.3992e-04)*phi*phi + (-0.0017197)*phi + (0.025335));
}
if(sec == 4){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 4] is:
    dp = dp + ((-4.0246e-06)*phi*phi + (3.4630e-05)*phi + (3.7794e-04))*pp*pp + ((6.5743e-05)*phi*phi + (-5.3345e-04)*phi + (-0.0063982))*pp + ((-2.1898e-04)*phi*phi + (0.0018233)*phi + (0.021071));
}
if(sec == 5){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 5] is:
    dp = dp + ((7.1270e-07)*phi*phi + (-2.6209e-05)*phi + (2.2981e-04))*pp*pp + ((-4.3241e-06)*phi*phi + (4.3945e-04)*phi + (-0.0046421))*pp + ((1.4313e-05)*phi*phi + (-0.0017363)*phi + (0.018327));
}
if(sec == 6){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 6] is:
    dp = dp + ((8.8226e-06)*phi*phi + (7.8655e-06)*phi + (-9.4725e-04))*pp*pp + ((-1.1813e-04)*phi*phi + (-1.0707e-04)*phi + (0.011478))*pp + ((3.7547e-04)*phi*phi + (3.8198e-04)*phi + (-0.030704));
}
"""

Output = """
if(sec == 1){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 1] is:
    dp = dp + ((1.0474e-05)*phi*phi + (-1.3817e-06)*phi + (-7.9929e-04))*pp*pp + ((-1.4636e-04)*phi*phi + (-7.3246e-05)*phi + (0.010204))*pp + ((5.0516e-04)*phi*phi + (6.8908e-04)*phi + (-0.029093));
}
if(sec == 2){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 2] is:
    dp = dp + ((-3.0529e-06)*phi*phi + (3.0917e-05)*phi + (-1.8985e-04))*pp*pp + ((4.9780e-05)*phi*phi + (-4.2868e-04)*phi + (0.002065))*pp + ((-1.7801e-04)*phi*phi + (0.0013342)*phi + (-0.0082156));
}
if(sec == 3){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 3] is:
    dp = dp + ((-8.6131e-06)*phi*phi + (-4.6774e-05)*phi + (5.3692e-04))*pp*pp + ((1.2284e-04)*phi*phi + (5.9110e-04)*phi + (-0.0083175))*pp + ((-3.7494e-04)*phi*phi + (-0.001433)*phi + (0.025335));
}
if(sec == 4){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 4] is:
    dp = dp + ((-2.7949e-06)*phi*phi + (2.8858e-05)*phi + (3.7794e-04))*pp*pp + ((4.5655e-05)*phi*phi + (-4.4454e-04)*phi + (-0.0063982))*pp + ((-1.5207e-04)*phi*phi + (0.0015194)*phi + (0.021071));
}
if(sec == 5){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 5] is:
    dp = dp + ((4.9493e-07)*phi*phi + (-2.1841e-05)*phi + (2.2981e-04))*pp*pp + ((-3.0029e-06)*phi*phi + (3.6620e-04)*phi + (-0.0046421))*pp + ((9.9398e-06)*phi*phi + (-0.0014469)*phi + (0.018327));
}
if(sec == 6){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 6] is:
    dp = dp + ((6.1268e-06)*phi*phi + (6.5546e-06)*phi + (-9.4725e-04))*pp*pp + ((-8.2037e-05)*phi*phi + (-8.9222e-05)*phi + (0.011478))*pp + ((2.6074e-04)*phi*phi + (3.1832e-04)*phi + (-0.030704));
}
"""


Output = """
if(sec == 1){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 1] is:
    dp = dp + ((-7.3331e-06)*phi*phi + (2.9512e-06)*phi + (1.4857e-04))*pp*pp + ((9.4041e-05)*phi*phi + (5.5267e-05)*phi + (-5.8894e-04))*pp + ((-3.0270e-04)*phi*phi + (-5.5944e-04)*phi + (-0.0034499));
}
if(sec == 2){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 2] is:
    dp = dp + ((1.0596e-06)*phi*phi + (3.3391e-05)*phi + (3.6760e-05))*pp*pp + ((-1.8672e-05)*phi*phi + (-4.6240e-04)*phi + (1.1258e-04))*pp + ((8.9102e-05)*phi*phi + (0.0014821)*phi + (-0.0025893));
}
if(sec == 3){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 3] is:
    dp = dp + ((-8.6131e-06)*phi*phi + (-4.6774e-05)*phi + (5.3692e-04))*pp*pp + ((1.2284e-04)*phi*phi + (5.9110e-04)*phi + (-0.0083175))*pp + ((-3.7494e-04)*phi*phi + (-0.001433)*phi + (0.025335));
}
if(sec == 4){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 4] is:
    dp = dp + ((-2.7949e-06)*phi*phi + (2.8858e-05)*phi + (3.7794e-04))*pp*pp + ((4.5655e-05)*phi*phi + (-4.4454e-04)*phi + (-0.0063982))*pp + ((-1.5207e-04)*phi*phi + (0.0015194)*phi + (0.021071));
}
if(sec == 5){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 5] is:
    dp = dp + ((4.9493e-07)*phi*phi + (-2.1841e-05)*phi + (2.2981e-04))*pp*pp + ((-3.0029e-06)*phi*phi + (3.6620e-04)*phi + (-0.0046421))*pp + ((9.9398e-06)*phi*phi + (-0.0014469)*phi + (0.018327));
}
if(sec == 6){
    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector 6] is:
    dp = dp + ((6.1268e-06)*phi*phi + (6.5546e-06)*phi + (-9.4725e-04))*pp*pp + ((-8.2037e-05)*phi*phi + (-8.9222e-05)*phi + (0.011478))*pp + ((2.6074e-04)*phi*phi + (3.1832e-04)*phi + (-0.030704));
}
"""


Output = Output.replace("dp +", "    ")
Output = Output.replace("\n    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Full Elastic)][Sector ", "")
Output = Output.replace("\n    // The CONTINUOUS QUADRATIC function predicted for ∆p_{El} for [Cor = Corrected (Without Pion)][Sector ", "")
Output = Output.replace("\n    // The CONTINUOUS QUADRATIC function predicted for ∆p_{#pi^{+}} for [Cor = Corrected (Full Elastic)][Sector ", "")
for sec in ["1", "2", "3", "4", "5", "6"]:
    Output = Output.replace("".join([sec, "] is"]), "")
    Output = Output.replace("".join(["sec == ", sec]), "".join(["'", sec, "' in str(sec)"]))
    
Output = Output.replace("{", "")
Output = Output.replace("}", "")
Output = Output.replace(";", "")
Output = Output.replace("\n\n", "\n")
print(Output)


if('1' in str(sec)):
    dp =      ((-7.3331e-06)*phi*phi + (2.9512e-06)*phi + (1.4857e-04))*pp*pp + ((9.4041e-05)*phi*phi + (5.5267e-05)*phi + (-5.8894e-04))*pp + ((-3.0270e-04)*phi*phi + (-5.5944e-04)*phi + (-0.0034499))
if('2' in str(sec)):
    dp =      ((1.0596e-06)*phi*phi + (3.3391e-05)*phi + (3.6760e-05))*pp*pp + ((-1.8672e-05)*phi*phi + (-4.6240e-04)*phi + (1.1258e-04))*pp + ((8.9102e-05)*phi*phi + (0.0014821)*phi + (-0.0025893))
if('3' in str(sec)):
    dp =      ((-8.6131e-06)*phi*phi + (-4.6774e-05)*phi + (5.3692e-04))*pp*pp + ((1.2284e-04)*phi*phi + (5.9110e-04)*phi + (-0.0083175))*pp + ((-3.7494e-04)*phi*phi + (-0.001433)*phi + (0.025335))
if('4' in str(sec)):
    dp =      ((-2.7949e-06)*phi*phi + (2.8858e-05)*phi + (3.7794e-04))*pp*pp + ((4.5655e-05)*phi*phi + (-4.4454e-04)*phi + (-0.0063982))*pp + ((-1.5207e-04)*phi*phi + (0.0015194)*phi + (0.021071))
if('5' in str(sec)):
    dp =      ((4.9493e-07)*phi*phi + (-2.1841e-05)*phi + (2.2981e-04))*pp*pp + ((-3.0029e-06)*ph

In [13]:
Output = str("""
if(sec == 1):
    if(phi == 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (-0.0275)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (-0.0075)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (0.0075)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (0.0066299)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (0.0058488)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (0.0075)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (0.0062714)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (-0.0025)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (0.0060532)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (-0.0037815)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (-0.005277)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (-0.0060711)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (-0.0050388)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (-0.0045939)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (0.0075)

}


    if(phi < 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (0.0095565)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (0.0125)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (0.0175)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (0.0075)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (0.0075)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (0.0075)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (0.0075)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (0.0075)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (-0.0025)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (-0.0039644)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (-0.0052809)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (-0.0062263)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (0.0046)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (0.0075)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (0.0075)

}


    if(phi > 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (0.0325)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (0.0125)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (0.027105)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (0.021952)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (0.0175)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (0.0075)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (0.0075)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (0.0075)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (0.0072161)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (0.0051469)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (0.0033626)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (0.00214)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (0.0016728)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (-0.0038227)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (0.0075)

}


if(sec == 2):
    if(phi == 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (-0.011201)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (-0.0075)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (0.0075)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (-0.0025)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (-0.0025)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (0.0026875)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (-0.0025)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (-0.0025)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (-0.0053608)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (-0.0057533)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (-0.0063951)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (-0.0076477)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (-0.0025)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (-0.0025)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (-0.0088821)

}


    if(phi < 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (-0.0275)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (-0.01383)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (-0.0125)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (-0.0025)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (-0.0125)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (-0.0025)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (-0.0032749)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (-0.0045748)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (-0.0041561)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (0.0044033)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (0.0038682)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (0.0035534)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (0.0033245)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (7.4302e-04)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (-0.0087925)

}


    if(phi > 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (-0.0075)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (-0.0075)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (-0.0125)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (-0.0025)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (-0.0071731)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (-0.0025037)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (-0.0057927)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (-0.0049888)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (-0.0048181)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (-0.0039585)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (0.0017416)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (0.0022288)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (0.0023434)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (0.0015866)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (-0.0098193)

}


if(sec == 3):
    if(phi == 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (0.0125)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (0.0085081)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (-0.0025)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (-0.0025)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (-0.0025)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (0.0042995)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (-0.0025)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (-0.0025)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (-0.0055132)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (-0.0062554)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (-0.0072941)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (-0.0071204)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (-0.0067798)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (-0.0068743)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (-0.010719)

}


    if(phi < 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (-0.0095851)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (-0.0087583)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (-0.0025)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (-0.0044901)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (-0.0026287)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (0.0023166)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (-0.0025)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (-0.004037)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (-0.0052233)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (-0.0062223)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (-0.0076923)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (-0.0050494)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (0.0039122)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (0.0040091)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (-0.010973)

}


    if(phi > 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (-0.0175)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (-0.0075)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (-0.0025)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (-0.0035383)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (-0.0025)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (0.0060647)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (0.0057439)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (0.0075)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (0.006724)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (0.0052542)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (0.00585)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (0.0073739)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (0.0075)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (0.0052166)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (-0.0095815)

}


if(sec == 4):
    if(phi == 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (0.0025)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (0.0125)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (0.0025)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (0.002786)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (0.0065305)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (-0.0025)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (-0.0025)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (-0.0031312)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (-0.0036763)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (-0.0084296)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (-0.0068869)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (-0.0057889)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (-0.0041669)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (-0.0053537)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (-0.0094853)

}


    if(phi < 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (-0.0175)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (0.003607)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (0.0043724)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (-0.0093219)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (-0.0088988)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (0.0034191)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (-0.0028651)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (-0.0025)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (0.0013609)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (-0.0039868)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (0.0030688)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (0.0039286)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (0.0054803)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (0.0075)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (-0.0086098)

}


    if(phi > 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (0.010794)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (0.0019473)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (0.0046589)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (0.0075)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (-0.0025)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (-0.0025)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (-0.0025)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (-0.0025)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (-0.0025)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (-0.0042839)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (-0.0048178)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (0.0034608)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (0.0074693)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (-0.0062123)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (-0.01103)

}


if(sec == 5):
    if(phi == 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (0.0125)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (0.011741)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (-0.0025)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (-0.0025)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (0.004311)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (0.0037247)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (0.0033987)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (0.0025726)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (-0.0037541)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (-0.0070524)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (-0.0079562)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (0.0032739)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (0.0045375)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (-0.0077872)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (-0.0125)

}


    if(phi < 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (0.029198)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (0.0225)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (0.0075)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (-0.0081394)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (0.0034255)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (-0.0056706)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (0.0047587)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (0.0028933)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (0.0059879)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (0.0032886)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (0.0034665)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (0.0048434)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (0.0049025)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (-0.0068684)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (-0.0125)

}


    if(phi > 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (0.0025)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (0.0056178)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (0.0079325)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (-0.0053782)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (-0.0061106)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (-0.0089871)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (-0.0034898)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (-0.0053093)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (0.002401)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (0.0024395)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (0.0058828)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (0.0075)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (0.0075)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (0.0013209)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (-0.0125)

}


if(sec == 6):
    if(phi == 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (-0.010189)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (-0.0095743)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (-0.0075)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (0.0051125)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (0.0035578)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (0.0053807)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (0.0050289)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (0.0051492)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (0.004109)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (0.0034371)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (-0.0077106)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (9.6011e-04)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (0.0021057)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (-0.0025)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (-0.0125)

}


    if(phi < 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (0.010664)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (0.0077653)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (-0.0025)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (5.6305e-04)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (0.0036133)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (-0.0034767)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (-0.0025)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (0.0025357)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (0.0024886)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (0.0022)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (0.006382)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (0.0075)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (0.0075)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (0.0021513)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (-0.018327)

}


    if(phi > 0):
        if(2.0 < pp and pp < 2.5):
            dp = dp + (0.0125)
        if(2.5 < pp and pp < 3.0):
            dp = dp + (0.0042064)
        if(3.0 < pp and pp < 3.5):
            dp = dp + (0.0075)
        if(3.5 < pp and pp < 4.0):
            dp = dp + (-0.0025)
        if(4.0 < pp and pp < 4.5):
            dp = dp + (0.0028824)
        if(4.5 < pp and pp < 5.0):
            dp = dp + (0.0075)
        if(5.0 < pp and pp < 5.5):
            dp = dp + (0.0069014)
        if(5.5 < pp and pp < 6.0):
            dp = dp + (0.0049317)
        if(6.0 < pp and pp < 6.5):
            dp = dp + (0.003036)
        if(6.5 < pp and pp < 7.0):
            dp = dp + (-0.0049006)
        if(7.0 < pp and pp < 7.5):
            dp = dp + (0.0030817)
        if(7.5 < pp and pp < 8.0):
            dp = dp + (0.0024117)
        if(8.0 < pp and pp < 8.5):
            dp = dp + (0.0034605)
        if(8.5 < pp and pp < 9.0):
            dp = dp + (0.0027162)
        if(9.0 < pp and pp < 9.5):
            dp = dp + (-0.00988)""".replace("\n}\n\n", "")).replace("dp = dp + ", "dp1 = ")

for sec in ["1", "2", "3", "4", "5", "6"]:
    Output = Output.replace("".join(["sec == ", sec]), "".join(["'", sec, "' in str(sec)"]))
    
print(Output)


if('1' in str(sec)):
    if(phi == 0):
        if(2.0 < pp and pp < 2.5):
            dp1 = (-0.0275)
        if(2.5 < pp and pp < 3.0):
            dp1 = (-0.0075)
        if(3.0 < pp and pp < 3.5):
            dp1 = (0.0075)
        if(3.5 < pp and pp < 4.0):
            dp1 = (0.0066299)
        if(4.0 < pp and pp < 4.5):
            dp1 = (0.0058488)
        if(4.5 < pp and pp < 5.0):
            dp1 = (0.0075)
        if(5.0 < pp and pp < 5.5):
            dp1 = (0.0062714)
        if(5.5 < pp and pp < 6.0):
            dp1 = (-0.0025)
        if(6.0 < pp and pp < 6.5):
            dp1 = (0.0060532)
        if(6.5 < pp and pp < 7.0):
            dp1 = (-0.0037815)
        if(7.0 < pp and pp < 7.5):
            dp1 = (-0.005277)
        if(7.5 < pp and pp < 8.0):
            dp1 = (-0.0060711)
        if(8.0 < pp and pp < 8.5):
            dp1 = (-0.0050388)
        if(8.5 < pp and pp < 9.0):
            dp1 = (-0.0045939)
        if(9.0 < pp and pp < 9.5):
            dp1 = (0.00